# Import Packages and Files

In [1]:
import torch.nn as nn
from functionalities import dataloader as dl
from functionalities import evaluater as ev
from functionalities import filemanager as fm
from functionalities import trainer as tr
from functionalities import plot as p
from architecture import RotNet as RN

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 128)

# Initialize Loss Criterion

In [4]:
criterion = nn.CrossEntropyLoss()

# Train RotNet for Rotation Task and Classifiers on Feature Maps 

In [5]:
# set rot classes
rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']

## 3 Block RotNet

In [12]:
# initialize network
net_block3 = RN.RotNet(num_classes=4, num_conv_block=3, add_avg_pool=False)

In [13]:
# train network
rot_block3_loss_log, _, rot_block3_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block3, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

[1, 60] loss: 1.141
[1, 120] loss: 0.999
[1, 180] loss: 0.920
[1, 240] loss: 0.851
[1, 300] loss: 0.790
[1, 360] loss: 0.759
Epoch: 1 -> Loss: 0.787678480148
Epoch: 1 -> Test Accuracy: 69.21
[2, 60] loss: 0.701
[2, 120] loss: 0.696
[2, 180] loss: 0.677
[2, 240] loss: 0.648
[2, 300] loss: 0.647
[2, 360] loss: 0.616
Epoch: 2 -> Loss: 0.662011623383
Epoch: 2 -> Test Accuracy: 76.055
[3, 60] loss: 0.596
[3, 120] loss: 0.581
[3, 180] loss: 0.577
[3, 240] loss: 0.583
[3, 300] loss: 0.557
[3, 360] loss: 0.566
Epoch: 3 -> Loss: 0.581205248833
Epoch: 3 -> Test Accuracy: 78.8075
[4, 60] loss: 0.544
[4, 120] loss: 0.519
[4, 180] loss: 0.517
[4, 240] loss: 0.537
[4, 300] loss: 0.507
[4, 360] loss: 0.507
Epoch: 4 -> Loss: 0.558216452599
Epoch: 4 -> Test Accuracy: 79.01
[5, 60] loss: 0.496
[5, 120] loss: 0.497
[5, 180] loss: 0.486
[5, 240] loss: 0.495
[5, 300] loss: 0.473
[5, 360] loss: 0.472
Epoch: 5 -> Loss: 0.537355840206
Epoch: 5 -> Test Accuracy: 81.725
[6, 60] loss: 0.449
[6, 120] loss: 0.464


[42, 240] loss: 0.322
[42, 300] loss: 0.311
[42, 360] loss: 0.313
Epoch: 42 -> Loss: 0.318148314953
Epoch: 42 -> Test Accuracy: 87.27
[43, 60] loss: 0.284
[43, 120] loss: 0.312
[43, 180] loss: 0.322
[43, 240] loss: 0.291
[43, 300] loss: 0.321
[43, 360] loss: 0.304
Epoch: 43 -> Loss: 0.382598012686
Epoch: 43 -> Test Accuracy: 83.9325
[44, 60] loss: 0.311
[44, 120] loss: 0.305
[44, 180] loss: 0.303
[44, 240] loss: 0.298
[44, 300] loss: 0.310
[44, 360] loss: 0.330
Epoch: 44 -> Loss: 0.220706671476
Epoch: 44 -> Test Accuracy: 87.075
[45, 60] loss: 0.298
[45, 120] loss: 0.296
[45, 180] loss: 0.313
[45, 240] loss: 0.317
[45, 300] loss: 0.302
[45, 360] loss: 0.308
Epoch: 45 -> Loss: 0.32049909234
Epoch: 45 -> Test Accuracy: 87.4575
[46, 60] loss: 0.290
[46, 120] loss: 0.306
[46, 180] loss: 0.302
[46, 240] loss: 0.307
[46, 300] loss: 0.309
[46, 360] loss: 0.316
Epoch: 46 -> Loss: 0.323618233204
Epoch: 46 -> Test Accuracy: 87.1525
[47, 60] loss: 0.294
[47, 120] loss: 0.285
[47, 180] loss: 0.300

[83, 240] loss: 0.145
[83, 300] loss: 0.153
[83, 360] loss: 0.158
Epoch: 83 -> Loss: 0.156742066145
Epoch: 83 -> Test Accuracy: 90.6825
[84, 60] loss: 0.135
[84, 120] loss: 0.139
[84, 180] loss: 0.137
[84, 240] loss: 0.145
[84, 300] loss: 0.158
[84, 360] loss: 0.149
Epoch: 84 -> Loss: 0.114132240415
Epoch: 84 -> Test Accuracy: 90.4775
[85, 60] loss: 0.129
[85, 120] loss: 0.129
[85, 180] loss: 0.137
[85, 240] loss: 0.145
[85, 300] loss: 0.148
[85, 360] loss: 0.152
Epoch: 85 -> Loss: 0.170121192932
Epoch: 85 -> Test Accuracy: 90.3325
[86, 60] loss: 0.132
[86, 120] loss: 0.145
[86, 180] loss: 0.141
[86, 240] loss: 0.146
[86, 300] loss: 0.144
[86, 360] loss: 0.155
Epoch: 86 -> Loss: 0.139373719692
Epoch: 86 -> Test Accuracy: 90.41
[87, 60] loss: 0.136
[87, 120] loss: 0.136
[87, 180] loss: 0.147
[87, 240] loss: 0.148
[87, 300] loss: 0.145
[87, 360] loss: 0.157
Epoch: 87 -> Loss: 0.234496861696
Epoch: 87 -> Test Accuracy: 89.7625
[88, 60] loss: 0.134
[88, 120] loss: 0.139
[88, 180] loss: 0.1

[123, 240] loss: 0.050
[123, 300] loss: 0.053
[123, 360] loss: 0.053
Epoch: 123 -> Loss: 0.0238902159035
Epoch: 123 -> Test Accuracy: 92.405
[124, 60] loss: 0.047
[124, 120] loss: 0.048
[124, 180] loss: 0.047
[124, 240] loss: 0.051
[124, 300] loss: 0.050
[124, 360] loss: 0.052
Epoch: 124 -> Loss: 0.0751750022173
Epoch: 124 -> Test Accuracy: 92.46
[125, 60] loss: 0.043
[125, 120] loss: 0.044
[125, 180] loss: 0.046
[125, 240] loss: 0.046
[125, 300] loss: 0.050
[125, 360] loss: 0.045
Epoch: 125 -> Loss: 0.0497423000634
Epoch: 125 -> Test Accuracy: 92.345
[126, 60] loss: 0.039
[126, 120] loss: 0.044
[126, 180] loss: 0.041
[126, 240] loss: 0.044
[126, 300] loss: 0.045
[126, 360] loss: 0.044
Epoch: 126 -> Loss: 0.0285705067217
Epoch: 126 -> Test Accuracy: 92.0975
[127, 60] loss: 0.040
[127, 120] loss: 0.041
[127, 180] loss: 0.038
[127, 240] loss: 0.040
[127, 300] loss: 0.042
[127, 360] loss: 0.044
Epoch: 127 -> Loss: 0.0293492916971
Epoch: 127 -> Test Accuracy: 92.2325
[128, 60] loss: 0.038


[162, 360] loss: 0.014
Epoch: 162 -> Loss: 0.0142389312387
Epoch: 162 -> Test Accuracy: 92.19
[163, 60] loss: 0.014
[163, 120] loss: 0.014
[163, 180] loss: 0.014
[163, 240] loss: 0.014
[163, 300] loss: 0.015
[163, 360] loss: 0.013
Epoch: 163 -> Loss: 0.0101729640737
Epoch: 163 -> Test Accuracy: 92.195
[164, 60] loss: 0.013
[164, 120] loss: 0.013
[164, 180] loss: 0.013
[164, 240] loss: 0.013
[164, 300] loss: 0.013
[164, 360] loss: 0.014
Epoch: 164 -> Loss: 0.0181327145547
Epoch: 164 -> Test Accuracy: 92.16
[165, 60] loss: 0.013
[165, 120] loss: 0.013
[165, 180] loss: 0.013
[165, 240] loss: 0.012
[165, 300] loss: 0.012
[165, 360] loss: 0.013
Epoch: 165 -> Loss: 0.00228249793872
Epoch: 165 -> Test Accuracy: 92.2325
[166, 60] loss: 0.013
[166, 120] loss: 0.012
[166, 180] loss: 0.012
[166, 240] loss: 0.013
[166, 300] loss: 0.013
[166, 360] loss: 0.012
Epoch: 166 -> Loss: 0.00735792284831
Epoch: 166 -> Test Accuracy: 92.18
[167, 60] loss: 0.012
[167, 120] loss: 0.011
[167, 180] loss: 0.013
[

In [14]:
# train NonLinearClassifiers on feature map of net_3block
block3_loss_log, _, block3_test_accuracy_log, _, _ = tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 100], 0.9, 5e-4, net_block3, criterion, trainloader, None, testloader) 

[1, 60] loss: 2.135
[1, 120] loss: 1.237
[1, 180] loss: 1.118
[1, 240] loss: 1.054
[1, 300] loss: 1.027
[1, 360] loss: 0.987
Epoch: 1 -> Loss: 1.02406382561
Epoch: 1 -> Test Accuracy: 68.73
[2, 60] loss: 0.908
[2, 120] loss: 0.900
[2, 180] loss: 0.892
[2, 240] loss: 0.870
[2, 300] loss: 0.870
[2, 360] loss: 0.830
Epoch: 2 -> Loss: 0.819397449493
Epoch: 2 -> Test Accuracy: 72.48
[3, 60] loss: 0.808
[3, 120] loss: 0.809
[3, 180] loss: 0.789
[3, 240] loss: 0.807
[3, 300] loss: 0.777
[3, 360] loss: 0.760
Epoch: 3 -> Loss: 0.705734670162
Epoch: 3 -> Test Accuracy: 74.34
[4, 60] loss: 0.757
[4, 120] loss: 0.763
[4, 180] loss: 0.744
[4, 240] loss: 0.729
[4, 300] loss: 0.732
[4, 360] loss: 0.722
Epoch: 4 -> Loss: 0.572060704231
Epoch: 4 -> Test Accuracy: 75.96
[5, 60] loss: 0.705
[5, 120] loss: 0.700
[5, 180] loss: 0.710
[5, 240] loss: 0.693
[5, 300] loss: 0.706
[5, 360] loss: 0.720
Epoch: 5 -> Loss: 0.530122101307
Epoch: 5 -> Test Accuracy: 76.62
[6, 60] loss: 0.681
[6, 120] loss: 0.698
[6, 1

Epoch: 42 -> Loss: 0.332994103432
Epoch: 42 -> Test Accuracy: 82.73
[43, 60] loss: 0.294
[43, 120] loss: 0.293
[43, 180] loss: 0.297
[43, 240] loss: 0.304
[43, 300] loss: 0.298
[43, 360] loss: 0.299
Epoch: 43 -> Loss: 0.326020866632
Epoch: 43 -> Test Accuracy: 83.09
[44, 60] loss: 0.295
[44, 120] loss: 0.292
[44, 180] loss: 0.283
[44, 240] loss: 0.294
[44, 300] loss: 0.293
[44, 360] loss: 0.282
Epoch: 44 -> Loss: 0.404775053263
Epoch: 44 -> Test Accuracy: 82.88
[45, 60] loss: 0.272
[45, 120] loss: 0.282
[45, 180] loss: 0.285
[45, 240] loss: 0.288
[45, 300] loss: 0.278
[45, 360] loss: 0.299
Epoch: 45 -> Loss: 0.15308393538
Epoch: 45 -> Test Accuracy: 82.92
[46, 60] loss: 0.275
[46, 120] loss: 0.268
[46, 180] loss: 0.283
[46, 240] loss: 0.278
[46, 300] loss: 0.278
[46, 360] loss: 0.270
Epoch: 46 -> Loss: 0.262244164944
Epoch: 46 -> Test Accuracy: 83.05
[47, 60] loss: 0.276
[47, 120] loss: 0.267
[47, 180] loss: 0.275
[47, 240] loss: 0.279
[47, 300] loss: 0.266
[47, 360] loss: 0.262
Epoch:

[84, 60] loss: 0.230
[84, 120] loss: 0.223
[84, 180] loss: 0.220
[84, 240] loss: 0.232
[84, 300] loss: 0.232
[84, 360] loss: 0.223
Epoch: 84 -> Loss: 0.275760382414
Epoch: 84 -> Test Accuracy: 83.1
[85, 60] loss: 0.224
[85, 120] loss: 0.223
[85, 180] loss: 0.217
[85, 240] loss: 0.229
[85, 300] loss: 0.232
[85, 360] loss: 0.239
Epoch: 85 -> Loss: 0.303332418203
Epoch: 85 -> Test Accuracy: 83.29
[86, 60] loss: 0.220
[86, 120] loss: 0.225
[86, 180] loss: 0.224
[86, 240] loss: 0.231
[86, 300] loss: 0.228
[86, 360] loss: 0.229
Epoch: 86 -> Loss: 0.361538261175
Epoch: 86 -> Test Accuracy: 83.19
[87, 60] loss: 0.221
[87, 120] loss: 0.224
[87, 180] loss: 0.223
[87, 240] loss: 0.225
[87, 300] loss: 0.224
[87, 360] loss: 0.215
Epoch: 87 -> Loss: 0.120859101415
Epoch: 87 -> Test Accuracy: 83.2
[88, 60] loss: 0.234
[88, 120] loss: 0.215
[88, 180] loss: 0.226
[88, 240] loss: 0.217
[88, 300] loss: 0.228
[88, 360] loss: 0.231
Epoch: 88 -> Loss: 0.202250763774
Epoch: 88 -> Test Accuracy: 83.17
[89, 60

[25, 360] loss: 0.228
Epoch: 25 -> Loss: 0.151886552572
Epoch: 25 -> Test Accuracy: 86.24
[26, 60] loss: 0.218
[26, 120] loss: 0.219
[26, 180] loss: 0.215
[26, 240] loss: 0.233
[26, 300] loss: 0.231
[26, 360] loss: 0.212
Epoch: 26 -> Loss: 0.146667078137
Epoch: 26 -> Test Accuracy: 86.05
[27, 60] loss: 0.197
[27, 120] loss: 0.212
[27, 180] loss: 0.211
[27, 240] loss: 0.213
[27, 300] loss: 0.216
[27, 360] loss: 0.221
Epoch: 27 -> Loss: 0.311142802238
Epoch: 27 -> Test Accuracy: 85.84
[28, 60] loss: 0.195
[28, 120] loss: 0.207
[28, 180] loss: 0.207
[28, 240] loss: 0.218
[28, 300] loss: 0.200
[28, 360] loss: 0.209
Epoch: 28 -> Loss: 0.142397254705
Epoch: 28 -> Test Accuracy: 85.58
[29, 60] loss: 0.192
[29, 120] loss: 0.209
[29, 180] loss: 0.199
[29, 240] loss: 0.206
[29, 300] loss: 0.199
[29, 360] loss: 0.204
Epoch: 29 -> Loss: 0.286255061626
Epoch: 29 -> Test Accuracy: 85.57
[30, 60] loss: 0.196
[30, 120] loss: 0.197
[30, 180] loss: 0.198
[30, 240] loss: 0.204
[30, 300] loss: 0.196
[30, 

Epoch: 66 -> Test Accuracy: 86.49
[67, 60] loss: 0.090
[67, 120] loss: 0.089
[67, 180] loss: 0.094
[67, 240] loss: 0.091
[67, 300] loss: 0.091
[67, 360] loss: 0.097
Epoch: 67 -> Loss: 0.0722390115261
Epoch: 67 -> Test Accuracy: 86.52
[68, 60] loss: 0.093
[68, 120] loss: 0.090
[68, 180] loss: 0.093
[68, 240] loss: 0.095
[68, 300] loss: 0.094
[68, 360] loss: 0.097
Epoch: 68 -> Loss: 0.0976048186421
Epoch: 68 -> Test Accuracy: 86.5
[69, 60] loss: 0.090
[69, 120] loss: 0.094
[69, 180] loss: 0.090
[69, 240] loss: 0.098
[69, 300] loss: 0.087
[69, 360] loss: 0.090
Epoch: 69 -> Loss: 0.124031342566
Epoch: 69 -> Test Accuracy: 86.45
[70, 60] loss: 0.094
[70, 120] loss: 0.088
[70, 180] loss: 0.090
[70, 240] loss: 0.087
[70, 300] loss: 0.089
[70, 360] loss: 0.091
Epoch: 70 -> Loss: 0.0749558657408
Epoch: 70 -> Test Accuracy: 86.42
[71, 60] loss: 0.092
[71, 120] loss: 0.093
[71, 180] loss: 0.086
[71, 240] loss: 0.088
[71, 300] loss: 0.084
[71, 360] loss: 0.094
Epoch: 71 -> Loss: 0.264926105738
Epo

[8, 180] loss: 1.476
[8, 240] loss: 1.464
[8, 300] loss: 1.475
[8, 360] loss: 1.473
Epoch: 8 -> Loss: 1.44293642044
Epoch: 8 -> Test Accuracy: 43.53
[9, 60] loss: 1.465
[9, 120] loss: 1.465
[9, 180] loss: 1.478
[9, 240] loss: 1.463
[9, 300] loss: 1.478
[9, 360] loss: 1.462
Epoch: 9 -> Loss: 1.58194768429
Epoch: 9 -> Test Accuracy: 44.31
[10, 60] loss: 1.480
[10, 120] loss: 1.461
[10, 180] loss: 1.447
[10, 240] loss: 1.470
[10, 300] loss: 1.470
[10, 360] loss: 1.457
Epoch: 10 -> Loss: 1.58967161179
Epoch: 10 -> Test Accuracy: 44.39
[11, 60] loss: 1.453
[11, 120] loss: 1.458
[11, 180] loss: 1.471
[11, 240] loss: 1.458
[11, 300] loss: 1.453
[11, 360] loss: 1.481
Epoch: 11 -> Loss: 1.41367149353
Epoch: 11 -> Test Accuracy: 43.93
[12, 60] loss: 1.469
[12, 120] loss: 1.442
[12, 180] loss: 1.453
[12, 240] loss: 1.462
[12, 300] loss: 1.472
[12, 360] loss: 1.448
Epoch: 12 -> Loss: 1.50912034512
Epoch: 12 -> Test Accuracy: 45.51
[13, 60] loss: 1.460
[13, 120] loss: 1.454
[13, 180] loss: 1.432
[1

Epoch: 49 -> Test Accuracy: 53.06
[50, 60] loss: 1.152
[50, 120] loss: 1.154
[50, 180] loss: 1.148
[50, 240] loss: 1.178
[50, 300] loss: 1.159
[50, 360] loss: 1.159
Epoch: 50 -> Loss: 1.01887559891
Epoch: 50 -> Test Accuracy: 53.1
[51, 60] loss: 1.158
[51, 120] loss: 1.145
[51, 180] loss: 1.157
[51, 240] loss: 1.177
[51, 300] loss: 1.156
[51, 360] loss: 1.163
Epoch: 51 -> Loss: 1.36545753479
Epoch: 51 -> Test Accuracy: 53.01
[52, 60] loss: 1.170
[52, 120] loss: 1.165
[52, 180] loss: 1.155
[52, 240] loss: 1.168
[52, 300] loss: 1.157
[52, 360] loss: 1.160
Epoch: 52 -> Loss: 1.01275980473
Epoch: 52 -> Test Accuracy: 53.23
[53, 60] loss: 1.151
[53, 120] loss: 1.154
[53, 180] loss: 1.165
[53, 240] loss: 1.142
[53, 300] loss: 1.161
[53, 360] loss: 1.172
Epoch: 53 -> Loss: 1.24386143684
Epoch: 53 -> Test Accuracy: 53.2
[54, 60] loss: 1.126
[54, 120] loss: 1.158
[54, 180] loss: 1.130
[54, 240] loss: 1.156
[54, 300] loss: 1.164
[54, 360] loss: 1.165
Epoch: 54 -> Loss: 1.104996562
Epoch: 54 -> T

[91, 240] loss: 1.140
[91, 300] loss: 1.108
[91, 360] loss: 1.129
Epoch: 91 -> Loss: 1.174051404
Epoch: 91 -> Test Accuracy: 53.87
[92, 60] loss: 1.144
[92, 120] loss: 1.129
[92, 180] loss: 1.132
[92, 240] loss: 1.123
[92, 300] loss: 1.128
[92, 360] loss: 1.119
Epoch: 92 -> Loss: 1.17889094353
Epoch: 92 -> Test Accuracy: 53.6
[93, 60] loss: 1.131
[93, 120] loss: 1.130
[93, 180] loss: 1.109
[93, 240] loss: 1.120
[93, 300] loss: 1.127
[93, 360] loss: 1.142
Epoch: 93 -> Loss: 0.920753359795
Epoch: 93 -> Test Accuracy: 53.92
[94, 60] loss: 1.133
[94, 120] loss: 1.138
[94, 180] loss: 1.126
[94, 240] loss: 1.136
[94, 300] loss: 1.139
[94, 360] loss: 1.125
Epoch: 94 -> Loss: 1.15011847019
Epoch: 94 -> Test Accuracy: 53.95
[95, 60] loss: 1.127
[95, 120] loss: 1.141
[95, 180] loss: 1.115
[95, 240] loss: 1.126
[95, 300] loss: 1.118
[95, 360] loss: 1.151
Epoch: 95 -> Loss: 1.27565062046
Epoch: 95 -> Test Accuracy: 54.02
[96, 60] loss: 1.117
[96, 120] loss: 1.108
[96, 180] loss: 1.125
[96, 240] lo

In [15]:
# train ConvClassifiers on feature map of net_3block
conv_block3_loss_log, _, conv_block3_test_accuracy_log, _, _ = tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], 
    [35, 70, 85, 100], 0.9, 5e-4, net_block3, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

[1, 60] loss: 1.371
[1, 120] loss: 1.023
[1, 180] loss: 0.913
[1, 240] loss: 0.887
[1, 300] loss: 0.824
[1, 360] loss: 0.783
Epoch: 1 -> Loss: 0.617035627365
Epoch: 1 -> Test Accuracy: 70.87
[2, 60] loss: 0.747
[2, 120] loss: 0.703
[2, 180] loss: 0.689
[2, 240] loss: 0.676
[2, 300] loss: 0.673
[2, 360] loss: 0.660
Epoch: 2 -> Loss: 0.749189436436
Epoch: 2 -> Test Accuracy: 74.43
[3, 60] loss: 0.627
[3, 120] loss: 0.612
[3, 180] loss: 0.619
[3, 240] loss: 0.611
[3, 300] loss: 0.628
[3, 360] loss: 0.587
Epoch: 3 -> Loss: 0.666461706161
Epoch: 3 -> Test Accuracy: 77.75
[4, 60] loss: 0.566
[4, 120] loss: 0.570
[4, 180] loss: 0.566
[4, 240] loss: 0.556
[4, 300] loss: 0.553
[4, 360] loss: 0.576
Epoch: 4 -> Loss: 0.634184956551
Epoch: 4 -> Test Accuracy: 77.39
[5, 60] loss: 0.521
[5, 120] loss: 0.541
[5, 180] loss: 0.559
[5, 240] loss: 0.535
[5, 300] loss: 0.520
[5, 360] loss: 0.528
Epoch: 5 -> Loss: 0.685723721981
Epoch: 5 -> Test Accuracy: 78.43
[6, 60] loss: 0.509
[6, 120] loss: 0.506
[6, 

Epoch: 42 -> Loss: 0.367429107428
Epoch: 42 -> Test Accuracy: 85.27
[43, 60] loss: 0.208
[43, 120] loss: 0.212
[43, 180] loss: 0.223
[43, 240] loss: 0.226
[43, 300] loss: 0.225
[43, 360] loss: 0.230
Epoch: 43 -> Loss: 0.246880248189
Epoch: 43 -> Test Accuracy: 86.26
[44, 60] loss: 0.211
[44, 120] loss: 0.199
[44, 180] loss: 0.218
[44, 240] loss: 0.218
[44, 300] loss: 0.224
[44, 360] loss: 0.239
Epoch: 44 -> Loss: 0.308743089437
Epoch: 44 -> Test Accuracy: 85.1
[45, 60] loss: 0.207
[45, 120] loss: 0.211
[45, 180] loss: 0.216
[45, 240] loss: 0.227
[45, 300] loss: 0.221
[45, 360] loss: 0.218
Epoch: 45 -> Loss: 0.193989947438
Epoch: 45 -> Test Accuracy: 84.7
[46, 60] loss: 0.211
[46, 120] loss: 0.204
[46, 180] loss: 0.213
[46, 240] loss: 0.218
[46, 300] loss: 0.221
[46, 360] loss: 0.237
Epoch: 46 -> Loss: 0.173785120249
Epoch: 46 -> Test Accuracy: 85.62
[47, 60] loss: 0.209
[47, 120] loss: 0.213
[47, 180] loss: 0.213
[47, 240] loss: 0.232
[47, 300] loss: 0.227
[47, 360] loss: 0.235
Epoch: 

[84, 60] loss: 0.093
[84, 120] loss: 0.098
[84, 180] loss: 0.095
[84, 240] loss: 0.098
[84, 300] loss: 0.099
[84, 360] loss: 0.100
Epoch: 84 -> Loss: 0.173047661781
Epoch: 84 -> Test Accuracy: 86.94
[85, 60] loss: 0.092
[85, 120] loss: 0.090
[85, 180] loss: 0.095
[85, 240] loss: 0.098
[85, 300] loss: 0.094
[85, 360] loss: 0.098
Epoch: 85 -> Loss: 0.0852631404996
Epoch: 85 -> Test Accuracy: 87.11
[86, 60] loss: 0.092
[86, 120] loss: 0.083
[86, 180] loss: 0.082
[86, 240] loss: 0.079
[86, 300] loss: 0.082
[86, 360] loss: 0.087
Epoch: 86 -> Loss: 0.117199338973
Epoch: 86 -> Test Accuracy: 87.23
[87, 60] loss: 0.080
[87, 120] loss: 0.077
[87, 180] loss: 0.080
[87, 240] loss: 0.082
[87, 300] loss: 0.080
[87, 360] loss: 0.081
Epoch: 87 -> Loss: 0.0730190724134
Epoch: 87 -> Test Accuracy: 87.05
[88, 60] loss: 0.080
[88, 120] loss: 0.079
[88, 180] loss: 0.082
[88, 240] loss: 0.077
[88, 300] loss: 0.079
[88, 360] loss: 0.077
Epoch: 88 -> Loss: 0.0719773620367
Epoch: 88 -> Test Accuracy: 87.09
[8

[25, 300] loss: 0.278
[25, 360] loss: 0.304
Epoch: 25 -> Loss: 0.273232907057
Epoch: 25 -> Test Accuracy: 85.32
[26, 60] loss: 0.250
[26, 120] loss: 0.254
[26, 180] loss: 0.270
[26, 240] loss: 0.270
[26, 300] loss: 0.286
[26, 360] loss: 0.270
Epoch: 26 -> Loss: 0.186416223645
Epoch: 26 -> Test Accuracy: 86.0
[27, 60] loss: 0.246
[27, 120] loss: 0.254
[27, 180] loss: 0.262
[27, 240] loss: 0.277
[27, 300] loss: 0.294
[27, 360] loss: 0.298
Epoch: 27 -> Loss: 0.315597355366
Epoch: 27 -> Test Accuracy: 85.52
[28, 60] loss: 0.252
[28, 120] loss: 0.255
[28, 180] loss: 0.255
[28, 240] loss: 0.277
[28, 300] loss: 0.280
[28, 360] loss: 0.274
Epoch: 28 -> Loss: 0.28716173768
Epoch: 28 -> Test Accuracy: 86.13
[29, 60] loss: 0.251
[29, 120] loss: 0.249
[29, 180] loss: 0.275
[29, 240] loss: 0.275
[29, 300] loss: 0.293
[29, 360] loss: 0.266
Epoch: 29 -> Loss: 0.236902907491
Epoch: 29 -> Test Accuracy: 86.14
[30, 60] loss: 0.242
[30, 120] loss: 0.254
[30, 180] loss: 0.268
[30, 240] loss: 0.270
[30, 30

Epoch: 66 -> Loss: 0.164013296366
Epoch: 66 -> Test Accuracy: 87.58
[67, 60] loss: 0.103
[67, 120] loss: 0.105
[67, 180] loss: 0.103
[67, 240] loss: 0.114
[67, 300] loss: 0.108
[67, 360] loss: 0.114
Epoch: 67 -> Loss: 0.0772556811571
Epoch: 67 -> Test Accuracy: 86.82
[68, 60] loss: 0.100
[68, 120] loss: 0.103
[68, 180] loss: 0.108
[68, 240] loss: 0.100
[68, 300] loss: 0.110
[68, 360] loss: 0.113
Epoch: 68 -> Loss: 0.0822162479162
Epoch: 68 -> Test Accuracy: 87.56
[69, 60] loss: 0.097
[69, 120] loss: 0.098
[69, 180] loss: 0.101
[69, 240] loss: 0.104
[69, 300] loss: 0.116
[69, 360] loss: 0.111
Epoch: 69 -> Loss: 0.102196291089
Epoch: 69 -> Test Accuracy: 87.01
[70, 60] loss: 0.100
[70, 120] loss: 0.093
[70, 180] loss: 0.108
[70, 240] loss: 0.109
[70, 300] loss: 0.104
[70, 360] loss: 0.114
Epoch: 70 -> Loss: 0.189903616905
Epoch: 70 -> Test Accuracy: 87.45
[71, 60] loss: 0.074
[71, 120] loss: 0.071
[71, 180] loss: 0.068
[71, 240] loss: 0.064
[71, 300] loss: 0.062
[71, 360] loss: 0.062
Epo

[8, 60] loss: 1.289
[8, 120] loss: 1.275
[8, 180] loss: 1.278
[8, 240] loss: 1.263
[8, 300] loss: 1.283
[8, 360] loss: 1.283
Epoch: 8 -> Loss: 1.37220048904
Epoch: 8 -> Test Accuracy: 50.36
[9, 60] loss: 1.268
[9, 120] loss: 1.257
[9, 180] loss: 1.257
[9, 240] loss: 1.270
[9, 300] loss: 1.275
[9, 360] loss: 1.271
Epoch: 9 -> Loss: 1.19314074516
Epoch: 9 -> Test Accuracy: 49.89
[10, 60] loss: 1.278
[10, 120] loss: 1.246
[10, 180] loss: 1.270
[10, 240] loss: 1.252
[10, 300] loss: 1.265
[10, 360] loss: 1.269
Epoch: 10 -> Loss: 1.18051970005
Epoch: 10 -> Test Accuracy: 50.67
[11, 60] loss: 1.248
[11, 120] loss: 1.282
[11, 180] loss: 1.244
[11, 240] loss: 1.267
[11, 300] loss: 1.260
[11, 360] loss: 1.241
Epoch: 11 -> Loss: 1.45489859581
Epoch: 11 -> Test Accuracy: 50.52
[12, 60] loss: 1.276
[12, 120] loss: 1.252
[12, 180] loss: 1.259
[12, 240] loss: 1.249
[12, 300] loss: 1.243
[12, 360] loss: 1.251
Epoch: 12 -> Loss: 1.29271221161
Epoch: 12 -> Test Accuracy: 49.86
[13, 60] loss: 1.252
[13, 

[49, 360] loss: 1.077
Epoch: 49 -> Loss: 1.06397080421
Epoch: 49 -> Test Accuracy: 55.96
[50, 60] loss: 1.067
[50, 120] loss: 1.063
[50, 180] loss: 1.069
[50, 240] loss: 1.073
[50, 300] loss: 1.082
[50, 360] loss: 1.074
Epoch: 50 -> Loss: 1.01605772972
Epoch: 50 -> Test Accuracy: 56.64
[51, 60] loss: 1.076
[51, 120] loss: 1.099
[51, 180] loss: 1.069
[51, 240] loss: 1.062
[51, 300] loss: 1.081
[51, 360] loss: 1.075
Epoch: 51 -> Loss: 0.962502360344
Epoch: 51 -> Test Accuracy: 56.47
[52, 60] loss: 1.090
[52, 120] loss: 1.079
[52, 180] loss: 1.045
[52, 240] loss: 1.064
[52, 300] loss: 1.083
[52, 360] loss: 1.047
Epoch: 52 -> Loss: 1.1834911108
Epoch: 52 -> Test Accuracy: 56.96
[53, 60] loss: 1.054
[53, 120] loss: 1.051
[53, 180] loss: 1.050
[53, 240] loss: 1.085
[53, 300] loss: 1.069
[53, 360] loss: 1.078
Epoch: 53 -> Loss: 1.04760217667
Epoch: 53 -> Test Accuracy: 57.01
[54, 60] loss: 1.051
[54, 120] loss: 1.065
[54, 180] loss: 1.095
[54, 240] loss: 1.070
[54, 300] loss: 1.064
[54, 360] 

[91, 60] loss: 0.897
[91, 120] loss: 0.905
[91, 180] loss: 0.898
[91, 240] loss: 0.892
[91, 300] loss: 0.913
[91, 360] loss: 0.901
Epoch: 91 -> Loss: 0.829767525196
Epoch: 91 -> Test Accuracy: 61.99
[92, 60] loss: 0.894
[92, 120] loss: 0.921
[92, 180] loss: 0.899
[92, 240] loss: 0.905
[92, 300] loss: 0.905
[92, 360] loss: 0.893
Epoch: 92 -> Loss: 1.0015989542
Epoch: 92 -> Test Accuracy: 61.87
[93, 60] loss: 0.892
[93, 120] loss: 0.907
[93, 180] loss: 0.910
[93, 240] loss: 0.913
[93, 300] loss: 0.886
[93, 360] loss: 0.907
Epoch: 93 -> Loss: 0.93932056427
Epoch: 93 -> Test Accuracy: 62.14
[94, 60] loss: 0.889
[94, 120] loss: 0.920
[94, 180] loss: 0.905
[94, 240] loss: 0.899
[94, 300] loss: 0.880
[94, 360] loss: 0.887
Epoch: 94 -> Loss: 1.04583060741
Epoch: 94 -> Test Accuracy: 61.96
[95, 60] loss: 0.905
[95, 120] loss: 0.894
[95, 180] loss: 0.888
[95, 240] loss: 0.920
[95, 300] loss: 0.913
[95, 360] loss: 0.884
Epoch: 95 -> Loss: 0.791828036308
Epoch: 95 -> Test Accuracy: 61.88
[96, 60] 

In [16]:
# save variables
fm.save_variable([rot_block3_loss_log, rot_block3_test_accuracy_log, 
                  block3_loss_log, block3_test_accuracy_log, 
                  conv_block3_loss_log, conv_block3_test_accuracy_log], "3_block_net")

In [17]:
# rename files
fm.add_block_to_name(3, [100, 200])

## 4 Block RotNet

In [18]:
# initialize network
net_block4 = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

In [19]:
# train network
rot_block4_loss_log, _, rot_block4_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block4, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

[1, 60] loss: 1.142
[1, 120] loss: 0.990
[1, 180] loss: 0.920
[1, 240] loss: 0.866
[1, 300] loss: 0.836
[1, 360] loss: 0.789
Epoch: 1 -> Loss: 0.781177341938
Epoch: 1 -> Test Accuracy: 68.8525
[2, 60] loss: 0.737
[2, 120] loss: 0.714
[2, 180] loss: 0.688
[2, 240] loss: 0.698
[2, 300] loss: 0.659
[2, 360] loss: 0.658
Epoch: 2 -> Loss: 0.628521740437
Epoch: 2 -> Test Accuracy: 75.445
[3, 60] loss: 0.617
[3, 120] loss: 0.613
[3, 180] loss: 0.601
[3, 240] loss: 0.577
[3, 300] loss: 0.589
[3, 360] loss: 0.572
Epoch: 3 -> Loss: 0.674859523773
Epoch: 3 -> Test Accuracy: 78.27
[4, 60] loss: 0.546
[4, 120] loss: 0.551
[4, 180] loss: 0.542
[4, 240] loss: 0.534
[4, 300] loss: 0.519
[4, 360] loss: 0.516
Epoch: 4 -> Loss: 0.378356218338
Epoch: 4 -> Test Accuracy: 79.4025
[5, 60] loss: 0.509
[5, 120] loss: 0.504
[5, 180] loss: 0.490
[5, 240] loss: 0.510
[5, 300] loss: 0.492
[5, 360] loss: 0.496
Epoch: 5 -> Loss: 0.567669272423
Epoch: 5 -> Test Accuracy: 80.6875
[6, 60] loss: 0.482
[6, 120] loss: 0.4

[42, 240] loss: 0.308
[42, 300] loss: 0.314
[42, 360] loss: 0.301
Epoch: 42 -> Loss: 0.282761156559
Epoch: 42 -> Test Accuracy: 86.685
[43, 60] loss: 0.302
[43, 120] loss: 0.300
[43, 180] loss: 0.313
[43, 240] loss: 0.313
[43, 300] loss: 0.297
[43, 360] loss: 0.314
Epoch: 43 -> Loss: 0.258620709181
Epoch: 43 -> Test Accuracy: 86.62
[44, 60] loss: 0.307
[44, 120] loss: 0.303
[44, 180] loss: 0.306
[44, 240] loss: 0.305
[44, 300] loss: 0.322
[44, 360] loss: 0.309
Epoch: 44 -> Loss: 0.460937410593
Epoch: 44 -> Test Accuracy: 86.7575
[45, 60] loss: 0.291
[45, 120] loss: 0.311
[45, 180] loss: 0.317
[45, 240] loss: 0.312
[45, 300] loss: 0.293
[45, 360] loss: 0.301
Epoch: 45 -> Loss: 0.43003988266
Epoch: 45 -> Test Accuracy: 85.46
[46, 60] loss: 0.300
[46, 120] loss: 0.294
[46, 180] loss: 0.306
[46, 240] loss: 0.312
[46, 300] loss: 0.304
[46, 360] loss: 0.295
Epoch: 46 -> Loss: 0.412907212973
Epoch: 46 -> Test Accuracy: 85.6225
[47, 60] loss: 0.306
[47, 120] loss: 0.292
[47, 180] loss: 0.300
[

[83, 240] loss: 0.148
[83, 300] loss: 0.159
[83, 360] loss: 0.150
Epoch: 83 -> Loss: 0.213200852275
Epoch: 83 -> Test Accuracy: 89.965
[84, 60] loss: 0.130
[84, 120] loss: 0.147
[84, 180] loss: 0.138
[84, 240] loss: 0.149
[84, 300] loss: 0.150
[84, 360] loss: 0.147
Epoch: 84 -> Loss: 0.190491721034
Epoch: 84 -> Test Accuracy: 89.765
[85, 60] loss: 0.135
[85, 120] loss: 0.148
[85, 180] loss: 0.141
[85, 240] loss: 0.143
[85, 300] loss: 0.148
[85, 360] loss: 0.146
Epoch: 85 -> Loss: 0.108416676521
Epoch: 85 -> Test Accuracy: 89.83
[86, 60] loss: 0.137
[86, 120] loss: 0.136
[86, 180] loss: 0.141
[86, 240] loss: 0.151
[86, 300] loss: 0.146
[86, 360] loss: 0.146
Epoch: 86 -> Loss: 0.170319244266
Epoch: 86 -> Test Accuracy: 90.21
[87, 60] loss: 0.138
[87, 120] loss: 0.138
[87, 180] loss: 0.143
[87, 240] loss: 0.156
[87, 300] loss: 0.148
[87, 360] loss: 0.148
Epoch: 87 -> Loss: 0.124249085784
Epoch: 87 -> Test Accuracy: 90.3175
[88, 60] loss: 0.125
[88, 120] loss: 0.139
[88, 180] loss: 0.138
[

[123, 240] loss: 0.050
[123, 300] loss: 0.048
[123, 360] loss: 0.047
Epoch: 123 -> Loss: 0.0314013287425
Epoch: 123 -> Test Accuracy: 91.6725
[124, 60] loss: 0.046
[124, 120] loss: 0.042
[124, 180] loss: 0.047
[124, 240] loss: 0.045
[124, 300] loss: 0.042
[124, 360] loss: 0.049
Epoch: 124 -> Loss: 0.0775769203901
Epoch: 124 -> Test Accuracy: 91.8775
[125, 60] loss: 0.041
[125, 120] loss: 0.038
[125, 180] loss: 0.044
[125, 240] loss: 0.044
[125, 300] loss: 0.041
[125, 360] loss: 0.043
Epoch: 125 -> Loss: 0.075447037816
Epoch: 125 -> Test Accuracy: 91.5125
[126, 60] loss: 0.036
[126, 120] loss: 0.036
[126, 180] loss: 0.037
[126, 240] loss: 0.038
[126, 300] loss: 0.041
[126, 360] loss: 0.041
Epoch: 126 -> Loss: 0.0122171742842
Epoch: 126 -> Test Accuracy: 91.8975
[127, 60] loss: 0.034
[127, 120] loss: 0.036
[127, 180] loss: 0.035
[127, 240] loss: 0.038
[127, 300] loss: 0.037
[127, 360] loss: 0.039
Epoch: 127 -> Loss: 0.0458883270621
Epoch: 127 -> Test Accuracy: 91.76
[128, 60] loss: 0.032

[162, 360] loss: 0.012
Epoch: 162 -> Loss: 0.0106612304226
Epoch: 162 -> Test Accuracy: 91.8225
[163, 60] loss: 0.010
[163, 120] loss: 0.012
[163, 180] loss: 0.011
[163, 240] loss: 0.011
[163, 300] loss: 0.010
[163, 360] loss: 0.011
Epoch: 163 -> Loss: 0.0206543244421
Epoch: 163 -> Test Accuracy: 91.82
[164, 60] loss: 0.009
[164, 120] loss: 0.010
[164, 180] loss: 0.010
[164, 240] loss: 0.010
[164, 300] loss: 0.010
[164, 360] loss: 0.011
Epoch: 164 -> Loss: 0.0153250638396
Epoch: 164 -> Test Accuracy: 91.81
[165, 60] loss: 0.009
[165, 120] loss: 0.010
[165, 180] loss: 0.009
[165, 240] loss: 0.009
[165, 300] loss: 0.009
[165, 360] loss: 0.010
Epoch: 165 -> Loss: 0.0122531848028
Epoch: 165 -> Test Accuracy: 91.8475
[166, 60] loss: 0.009
[166, 120] loss: 0.009
[166, 180] loss: 0.009
[166, 240] loss: 0.010
[166, 300] loss: 0.009
[166, 360] loss: 0.010
Epoch: 166 -> Loss: 0.00318783451803
Epoch: 166 -> Test Accuracy: 91.765
[167, 60] loss: 0.010
[167, 120] loss: 0.008
[167, 180] loss: 0.008


In [20]:
# train NonLinearClassifiers on feature map of net_3block
block4_loss_log, _, block4_test_accuracy_log, _, _ = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 100], 0.9, 5e-4, net_block4, criterion, trainloader, None, testloader) 

[1, 60] loss: 2.193
[1, 120] loss: 1.210
[1, 180] loss: 1.152
[1, 240] loss: 1.063
[1, 300] loss: 1.022
[1, 360] loss: 0.999
Epoch: 1 -> Loss: 0.855801939964
Epoch: 1 -> Test Accuracy: 66.44
[2, 60] loss: 0.934
[2, 120] loss: 0.945
[2, 180] loss: 0.883
[2, 240] loss: 0.879
[2, 300] loss: 0.904
[2, 360] loss: 0.872
Epoch: 2 -> Loss: 0.800943017006
Epoch: 2 -> Test Accuracy: 70.93
[3, 60] loss: 0.831
[3, 120] loss: 0.826
[3, 180] loss: 0.833
[3, 240] loss: 0.793
[3, 300] loss: 0.806
[3, 360] loss: 0.801
Epoch: 3 -> Loss: 0.897849202156
Epoch: 3 -> Test Accuracy: 73.18
[4, 60] loss: 0.782
[4, 120] loss: 0.751
[4, 180] loss: 0.758
[4, 240] loss: 0.797
[4, 300] loss: 0.741
[4, 360] loss: 0.754
Epoch: 4 -> Loss: 0.613576591015
Epoch: 4 -> Test Accuracy: 74.4
[5, 60] loss: 0.734
[5, 120] loss: 0.731
[5, 180] loss: 0.704
[5, 240] loss: 0.724
[5, 300] loss: 0.723
[5, 360] loss: 0.738
Epoch: 5 -> Loss: 0.856733620167
Epoch: 5 -> Test Accuracy: 75.34
[6, 60] loss: 0.708
[6, 120] loss: 0.696
[6, 1

Epoch: 42 -> Loss: 0.201099067926
Epoch: 42 -> Test Accuracy: 82.4
[43, 60] loss: 0.331
[43, 120] loss: 0.310
[43, 180] loss: 0.319
[43, 240] loss: 0.315
[43, 300] loss: 0.312
[43, 360] loss: 0.317
Epoch: 43 -> Loss: 0.307905286551
Epoch: 43 -> Test Accuracy: 82.69
[44, 60] loss: 0.312
[44, 120] loss: 0.300
[44, 180] loss: 0.306
[44, 240] loss: 0.309
[44, 300] loss: 0.298
[44, 360] loss: 0.309
Epoch: 44 -> Loss: 0.371604979038
Epoch: 44 -> Test Accuracy: 82.74
[45, 60] loss: 0.303
[45, 120] loss: 0.297
[45, 180] loss: 0.297
[45, 240] loss: 0.295
[45, 300] loss: 0.310
[45, 360] loss: 0.315
Epoch: 45 -> Loss: 0.396926254034
Epoch: 45 -> Test Accuracy: 82.61
[46, 60] loss: 0.288
[46, 120] loss: 0.295
[46, 180] loss: 0.273
[46, 240] loss: 0.288
[46, 300] loss: 0.293
[46, 360] loss: 0.291
Epoch: 46 -> Loss: 0.280866056681
Epoch: 46 -> Test Accuracy: 82.7
[47, 60] loss: 0.283
[47, 120] loss: 0.296
[47, 180] loss: 0.297
[47, 240] loss: 0.280
[47, 300] loss: 0.288
[47, 360] loss: 0.273
Epoch: 

[84, 60] loss: 0.245
[84, 120] loss: 0.240
[84, 180] loss: 0.246
[84, 240] loss: 0.237
[84, 300] loss: 0.240
[84, 360] loss: 0.245
Epoch: 84 -> Loss: 0.322443187237
Epoch: 84 -> Test Accuracy: 82.71
[85, 60] loss: 0.242
[85, 120] loss: 0.249
[85, 180] loss: 0.241
[85, 240] loss: 0.252
[85, 300] loss: 0.243
[85, 360] loss: 0.227
Epoch: 85 -> Loss: 0.196157410741
Epoch: 85 -> Test Accuracy: 82.66
[86, 60] loss: 0.239
[86, 120] loss: 0.238
[86, 180] loss: 0.239
[86, 240] loss: 0.248
[86, 300] loss: 0.239
[86, 360] loss: 0.235
Epoch: 86 -> Loss: 0.204234689474
Epoch: 86 -> Test Accuracy: 82.67
[87, 60] loss: 0.231
[87, 120] loss: 0.247
[87, 180] loss: 0.239
[87, 240] loss: 0.235
[87, 300] loss: 0.240
[87, 360] loss: 0.242
Epoch: 87 -> Loss: 0.325607389212
Epoch: 87 -> Test Accuracy: 82.73
[88, 60] loss: 0.239
[88, 120] loss: 0.241
[88, 180] loss: 0.244
[88, 240] loss: 0.244
[88, 300] loss: 0.235
[88, 360] loss: 0.242
Epoch: 88 -> Loss: 0.138225629926
Epoch: 88 -> Test Accuracy: 82.77
[89, 

[25, 360] loss: 0.222
Epoch: 25 -> Loss: 0.26879543066
Epoch: 25 -> Test Accuracy: 85.06
[26, 60] loss: 0.209
[26, 120] loss: 0.226
[26, 180] loss: 0.206
[26, 240] loss: 0.213
[26, 300] loss: 0.221
[26, 360] loss: 0.232
Epoch: 26 -> Loss: 0.196099326015
Epoch: 26 -> Test Accuracy: 85.03
[27, 60] loss: 0.198
[27, 120] loss: 0.213
[27, 180] loss: 0.212
[27, 240] loss: 0.207
[27, 300] loss: 0.211
[27, 360] loss: 0.213
Epoch: 27 -> Loss: 0.232765719295
Epoch: 27 -> Test Accuracy: 85.08
[28, 60] loss: 0.197
[28, 120] loss: 0.196
[28, 180] loss: 0.206
[28, 240] loss: 0.219
[28, 300] loss: 0.209
[28, 360] loss: 0.215
Epoch: 28 -> Loss: 0.136992484331
Epoch: 28 -> Test Accuracy: 85.07
[29, 60] loss: 0.208
[29, 120] loss: 0.212
[29, 180] loss: 0.207
[29, 240] loss: 0.214
[29, 300] loss: 0.207
[29, 360] loss: 0.208
Epoch: 29 -> Loss: 0.233009338379
Epoch: 29 -> Test Accuracy: 85.23
[30, 60] loss: 0.196
[30, 120] loss: 0.187
[30, 180] loss: 0.211
[30, 240] loss: 0.216
[30, 300] loss: 0.202
[30, 3

Epoch: 66 -> Test Accuracy: 85.66
[67, 60] loss: 0.098
[67, 120] loss: 0.091
[67, 180] loss: 0.090
[67, 240] loss: 0.094
[67, 300] loss: 0.099
[67, 360] loss: 0.096
Epoch: 67 -> Loss: 0.0928951725364
Epoch: 67 -> Test Accuracy: 85.77
[68, 60] loss: 0.089
[68, 120] loss: 0.099
[68, 180] loss: 0.093
[68, 240] loss: 0.095
[68, 300] loss: 0.096
[68, 360] loss: 0.097
Epoch: 68 -> Loss: 0.0397040434182
Epoch: 68 -> Test Accuracy: 85.72
[69, 60] loss: 0.097
[69, 120] loss: 0.094
[69, 180] loss: 0.091
[69, 240] loss: 0.091
[69, 300] loss: 0.097
[69, 360] loss: 0.093
Epoch: 69 -> Loss: 0.196019023657
Epoch: 69 -> Test Accuracy: 85.72
[70, 60] loss: 0.094
[70, 120] loss: 0.093
[70, 180] loss: 0.098
[70, 240] loss: 0.096
[70, 300] loss: 0.087
[70, 360] loss: 0.097
Epoch: 70 -> Loss: 0.0682639628649
Epoch: 70 -> Test Accuracy: 85.58
[71, 60] loss: 0.088
[71, 120] loss: 0.089
[71, 180] loss: 0.093
[71, 240] loss: 0.090
[71, 300] loss: 0.099
[71, 360] loss: 0.093
Epoch: 71 -> Loss: 0.0670818313956
E

[8, 120] loss: 0.529
[8, 180] loss: 0.568
[8, 240] loss: 0.548
[8, 300] loss: 0.548
[8, 360] loss: 0.551
Epoch: 8 -> Loss: 0.607245922089
Epoch: 8 -> Test Accuracy: 76.85
[9, 60] loss: 0.528
[9, 120] loss: 0.545
[9, 180] loss: 0.552
[9, 240] loss: 0.532
[9, 300] loss: 0.539
[9, 360] loss: 0.550
Epoch: 9 -> Loss: 0.558493852615
Epoch: 9 -> Test Accuracy: 77.59
[10, 60] loss: 0.525
[10, 120] loss: 0.534
[10, 180] loss: 0.551
[10, 240] loss: 0.533
[10, 300] loss: 0.543
[10, 360] loss: 0.534
Epoch: 10 -> Loss: 0.50290876627
Epoch: 10 -> Test Accuracy: 77.13
[11, 60] loss: 0.522
[11, 120] loss: 0.535
[11, 180] loss: 0.535
[11, 240] loss: 0.537
[11, 300] loss: 0.524
[11, 360] loss: 0.535
Epoch: 11 -> Loss: 0.506026685238
Epoch: 11 -> Test Accuracy: 78.0
[12, 60] loss: 0.519
[12, 120] loss: 0.538
[12, 180] loss: 0.516
[12, 240] loss: 0.544
[12, 300] loss: 0.534
[12, 360] loss: 0.530
Epoch: 12 -> Loss: 0.4039940238
Epoch: 12 -> Test Accuracy: 77.09
[13, 60] loss: 0.490
[13, 120] loss: 0.531
[1

[49, 300] loss: 0.265
[49, 360] loss: 0.262
Epoch: 49 -> Loss: 0.285901218653
Epoch: 49 -> Test Accuracy: 80.86
[50, 60] loss: 0.265
[50, 120] loss: 0.264
[50, 180] loss: 0.255
[50, 240] loss: 0.261
[50, 300] loss: 0.269
[50, 360] loss: 0.262
Epoch: 50 -> Loss: 0.282857120037
Epoch: 50 -> Test Accuracy: 81.02
[51, 60] loss: 0.267
[51, 120] loss: 0.261
[51, 180] loss: 0.257
[51, 240] loss: 0.256
[51, 300] loss: 0.258
[51, 360] loss: 0.262
Epoch: 51 -> Loss: 0.238241270185
Epoch: 51 -> Test Accuracy: 81.13
[52, 60] loss: 0.256
[52, 120] loss: 0.254
[52, 180] loss: 0.269
[52, 240] loss: 0.262
[52, 300] loss: 0.265
[52, 360] loss: 0.269
Epoch: 52 -> Loss: 0.323007881641
Epoch: 52 -> Test Accuracy: 81.08
[53, 60] loss: 0.259
[53, 120] loss: 0.252
[53, 180] loss: 0.270
[53, 240] loss: 0.263
[53, 300] loss: 0.265
[53, 360] loss: 0.254
Epoch: 53 -> Loss: 0.248357981443
Epoch: 53 -> Test Accuracy: 81.08
[54, 60] loss: 0.255
[54, 120] loss: 0.250
[54, 180] loss: 0.259
[54, 240] loss: 0.265
[54, 

Epoch: 90 -> Loss: 0.103151679039
Epoch: 90 -> Test Accuracy: 81.03
[91, 60] loss: 0.232
[91, 120] loss: 0.218
[91, 180] loss: 0.219
[91, 240] loss: 0.217
[91, 300] loss: 0.227
[91, 360] loss: 0.224
Epoch: 91 -> Loss: 0.181434720755
Epoch: 91 -> Test Accuracy: 81.03
[92, 60] loss: 0.219
[92, 120] loss: 0.219
[92, 180] loss: 0.215
[92, 240] loss: 0.214
[92, 300] loss: 0.224
[92, 360] loss: 0.229
Epoch: 92 -> Loss: 0.398776292801
Epoch: 92 -> Test Accuracy: 81.03
[93, 60] loss: 0.211
[93, 120] loss: 0.225
[93, 180] loss: 0.221
[93, 240] loss: 0.226
[93, 300] loss: 0.210
[93, 360] loss: 0.223
Epoch: 93 -> Loss: 0.206774383783
Epoch: 93 -> Test Accuracy: 81.14
[94, 60] loss: 0.222
[94, 120] loss: 0.220
[94, 180] loss: 0.220
[94, 240] loss: 0.213
[94, 300] loss: 0.215
[94, 360] loss: 0.225
Epoch: 94 -> Loss: 0.2201487571
Epoch: 94 -> Test Accuracy: 81.08
[95, 60] loss: 0.208
[95, 120] loss: 0.214
[95, 180] loss: 0.226
[95, 240] loss: 0.224
[95, 300] loss: 0.224
[95, 360] loss: 0.219
Epoch: 

[32, 240] loss: 1.547
[32, 300] loss: 1.545
[32, 360] loss: 1.528
Epoch: 32 -> Loss: 1.68761444092
Epoch: 32 -> Test Accuracy: 39.89
[33, 60] loss: 1.525
[33, 120] loss: 1.537
[33, 180] loss: 1.544
[33, 240] loss: 1.517
[33, 300] loss: 1.535
[33, 360] loss: 1.534
Epoch: 33 -> Loss: 1.62631833553
Epoch: 33 -> Test Accuracy: 39.46
[34, 60] loss: 1.530
[34, 120] loss: 1.532
[34, 180] loss: 1.514
[34, 240] loss: 1.529
[34, 300] loss: 1.510
[34, 360] loss: 1.533
Epoch: 34 -> Loss: 1.60305440426
Epoch: 34 -> Test Accuracy: 38.84
[35, 60] loss: 1.527
[35, 120] loss: 1.520
[35, 180] loss: 1.524
[35, 240] loss: 1.515
[35, 300] loss: 1.540
[35, 360] loss: 1.532
Epoch: 35 -> Loss: 1.37478029728
Epoch: 35 -> Test Accuracy: 40.13
[36, 60] loss: 1.527
[36, 120] loss: 1.514
[36, 180] loss: 1.526
[36, 240] loss: 1.534
[36, 300] loss: 1.519
[36, 360] loss: 1.529
Epoch: 36 -> Loss: 1.59059739113
Epoch: 36 -> Test Accuracy: 39.49
[37, 60] loss: 1.505
[37, 120] loss: 1.533
[37, 180] loss: 1.555
[37, 240] 

Epoch: 73 -> Test Accuracy: 42.86
[74, 60] loss: 1.398
[74, 120] loss: 1.410
[74, 180] loss: 1.408
[74, 240] loss: 1.403
[74, 300] loss: 1.386
[74, 360] loss: 1.395
Epoch: 74 -> Loss: 1.50504624844
Epoch: 74 -> Test Accuracy: 42.6
[75, 60] loss: 1.406
[75, 120] loss: 1.394
[75, 180] loss: 1.406
[75, 240] loss: 1.406
[75, 300] loss: 1.394
[75, 360] loss: 1.401
Epoch: 75 -> Loss: 1.38171458244
Epoch: 75 -> Test Accuracy: 42.78
[76, 60] loss: 1.386
[76, 120] loss: 1.394
[76, 180] loss: 1.402
[76, 240] loss: 1.416
[76, 300] loss: 1.393
[76, 360] loss: 1.393
Epoch: 76 -> Loss: 1.5106703043
Epoch: 76 -> Test Accuracy: 42.8
[77, 60] loss: 1.398
[77, 120] loss: 1.383
[77, 180] loss: 1.393
[77, 240] loss: 1.400
[77, 300] loss: 1.401
[77, 360] loss: 1.405
Epoch: 77 -> Loss: 1.49361693859
Epoch: 77 -> Test Accuracy: 42.83
[78, 60] loss: 1.399
[78, 120] loss: 1.377
[78, 180] loss: 1.412
[78, 240] loss: 1.395
[78, 300] loss: 1.408
[78, 360] loss: 1.405
Epoch: 78 -> Loss: 1.49322938919
Epoch: 78 -> 

In [21]:
# train ConvClassifiers on feature map of net_3block
conv_block4_loss_log, _, conv_block4_test_accuracy_log, _, _ = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], 
    [35, 70, 85, 100], 0.9, 5e-4, net_block4, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

[1, 60] loss: 1.376
[1, 120] loss: 1.042
[1, 180] loss: 0.913
[1, 240] loss: 0.894
[1, 300] loss: 0.837
[1, 360] loss: 0.808
Epoch: 1 -> Loss: 0.741010427475
Epoch: 1 -> Test Accuracy: 69.36
[2, 60] loss: 0.738
[2, 120] loss: 0.732
[2, 180] loss: 0.716
[2, 240] loss: 0.703
[2, 300] loss: 0.695
[2, 360] loss: 0.686
Epoch: 2 -> Loss: 0.697795510292
Epoch: 2 -> Test Accuracy: 73.42
[3, 60] loss: 0.626
[3, 120] loss: 0.638
[3, 180] loss: 0.637
[3, 240] loss: 0.619
[3, 300] loss: 0.629
[3, 360] loss: 0.607
Epoch: 3 -> Loss: 0.611978173256
Epoch: 3 -> Test Accuracy: 74.3
[4, 60] loss: 0.567
[4, 120] loss: 0.581
[4, 180] loss: 0.598
[4, 240] loss: 0.586
[4, 300] loss: 0.587
[4, 360] loss: 0.582
Epoch: 4 -> Loss: 0.599568843842
Epoch: 4 -> Test Accuracy: 76.22
[5, 60] loss: 0.537
[5, 120] loss: 0.552
[5, 180] loss: 0.546
[5, 240] loss: 0.551
[5, 300] loss: 0.560
[5, 360] loss: 0.567
Epoch: 5 -> Loss: 0.653165221214
Epoch: 5 -> Test Accuracy: 76.33
[6, 60] loss: 0.529
[6, 120] loss: 0.512
[6, 1

Epoch: 42 -> Loss: 0.302528142929
Epoch: 42 -> Test Accuracy: 84.53
[43, 60] loss: 0.204
[43, 120] loss: 0.211
[43, 180] loss: 0.214
[43, 240] loss: 0.225
[43, 300] loss: 0.231
[43, 360] loss: 0.235
Epoch: 43 -> Loss: 0.222333043814
Epoch: 43 -> Test Accuracy: 83.7
[44, 60] loss: 0.221
[44, 120] loss: 0.218
[44, 180] loss: 0.217
[44, 240] loss: 0.221
[44, 300] loss: 0.238
[44, 360] loss: 0.237
Epoch: 44 -> Loss: 0.238920122385
Epoch: 44 -> Test Accuracy: 84.63
[45, 60] loss: 0.209
[45, 120] loss: 0.220
[45, 180] loss: 0.226
[45, 240] loss: 0.224
[45, 300] loss: 0.235
[45, 360] loss: 0.244
Epoch: 45 -> Loss: 0.263933122158
Epoch: 45 -> Test Accuracy: 84.81
[46, 60] loss: 0.206
[46, 120] loss: 0.223
[46, 180] loss: 0.224
[46, 240] loss: 0.234
[46, 300] loss: 0.219
[46, 360] loss: 0.245
Epoch: 46 -> Loss: 0.203419446945
Epoch: 46 -> Test Accuracy: 84.36
[47, 60] loss: 0.207
[47, 120] loss: 0.218
[47, 180] loss: 0.230
[47, 240] loss: 0.231
[47, 300] loss: 0.233
[47, 360] loss: 0.225
Epoch:

[84, 60] loss: 0.093
[84, 120] loss: 0.092
[84, 180] loss: 0.099
[84, 240] loss: 0.102
[84, 300] loss: 0.099
[84, 360] loss: 0.100
Epoch: 84 -> Loss: 0.0869622081518
Epoch: 84 -> Test Accuracy: 85.49
[85, 60] loss: 0.097
[85, 120] loss: 0.097
[85, 180] loss: 0.093
[85, 240] loss: 0.098
[85, 300] loss: 0.099
[85, 360] loss: 0.099
Epoch: 85 -> Loss: 0.100130841136
Epoch: 85 -> Test Accuracy: 85.39
[86, 60] loss: 0.088
[86, 120] loss: 0.091
[86, 180] loss: 0.084
[86, 240] loss: 0.089
[86, 300] loss: 0.083
[86, 360] loss: 0.086
Epoch: 86 -> Loss: 0.0797803252935
Epoch: 86 -> Test Accuracy: 85.53
[87, 60] loss: 0.087
[87, 120] loss: 0.087
[87, 180] loss: 0.082
[87, 240] loss: 0.083
[87, 300] loss: 0.086
[87, 360] loss: 0.082
Epoch: 87 -> Loss: 0.0665135756135
Epoch: 87 -> Test Accuracy: 85.44
[88, 60] loss: 0.079
[88, 120] loss: 0.082
[88, 180] loss: 0.083
[88, 240] loss: 0.084
[88, 300] loss: 0.082
[88, 360] loss: 0.082
Epoch: 88 -> Loss: 0.11388900131
Epoch: 88 -> Test Accuracy: 85.26
[89

[25, 300] loss: 0.284
[25, 360] loss: 0.279
Epoch: 25 -> Loss: 0.21403875947
Epoch: 25 -> Test Accuracy: 83.87
[26, 60] loss: 0.242
[26, 120] loss: 0.251
[26, 180] loss: 0.269
[26, 240] loss: 0.272
[26, 300] loss: 0.294
[26, 360] loss: 0.267
Epoch: 26 -> Loss: 0.328526705503
Epoch: 26 -> Test Accuracy: 83.57
[27, 60] loss: 0.248
[27, 120] loss: 0.248
[27, 180] loss: 0.258
[27, 240] loss: 0.275
[27, 300] loss: 0.270
[27, 360] loss: 0.276
Epoch: 27 -> Loss: 0.344846099615
Epoch: 27 -> Test Accuracy: 84.84
[28, 60] loss: 0.243
[28, 120] loss: 0.252
[28, 180] loss: 0.263
[28, 240] loss: 0.265
[28, 300] loss: 0.282
[28, 360] loss: 0.274
Epoch: 28 -> Loss: 0.290785968304
Epoch: 28 -> Test Accuracy: 84.18
[29, 60] loss: 0.248
[29, 120] loss: 0.246
[29, 180] loss: 0.254
[29, 240] loss: 0.267
[29, 300] loss: 0.285
[29, 360] loss: 0.268
Epoch: 29 -> Loss: 0.268285512924
Epoch: 29 -> Test Accuracy: 84.79
[30, 60] loss: 0.231
[30, 120] loss: 0.249
[30, 180] loss: 0.268
[30, 240] loss: 0.269
[30, 3

Epoch: 66 -> Loss: 0.0615716688335
Epoch: 66 -> Test Accuracy: 85.6
[67, 60] loss: 0.100
[67, 120] loss: 0.101
[67, 180] loss: 0.096
[67, 240] loss: 0.098
[67, 300] loss: 0.102
[67, 360] loss: 0.116
Epoch: 67 -> Loss: 0.120392024517
Epoch: 67 -> Test Accuracy: 85.41
[68, 60] loss: 0.098
[68, 120] loss: 0.095
[68, 180] loss: 0.104
[68, 240] loss: 0.098
[68, 300] loss: 0.106
[68, 360] loss: 0.121
Epoch: 68 -> Loss: 0.142064541578
Epoch: 68 -> Test Accuracy: 85.72
[69, 60] loss: 0.088
[69, 120] loss: 0.089
[69, 180] loss: 0.102
[69, 240] loss: 0.102
[69, 300] loss: 0.104
[69, 360] loss: 0.104
Epoch: 69 -> Loss: 0.106493912637
Epoch: 69 -> Test Accuracy: 86.29
[70, 60] loss: 0.108
[70, 120] loss: 0.101
[70, 180] loss: 0.102
[70, 240] loss: 0.109
[70, 300] loss: 0.111
[70, 360] loss: 0.104
Epoch: 70 -> Loss: 0.0716516599059
Epoch: 70 -> Test Accuracy: 85.79
[71, 60] loss: 0.072
[71, 120] loss: 0.070
[71, 180] loss: 0.065
[71, 240] loss: 0.065
[71, 300] loss: 0.064
[71, 360] loss: 0.063
Epoc

[8, 60] loss: 0.447
[8, 120] loss: 0.469
[8, 180] loss: 0.457
[8, 240] loss: 0.476
[8, 300] loss: 0.465
[8, 360] loss: 0.468
Epoch: 8 -> Loss: 0.547881960869
Epoch: 8 -> Test Accuracy: 78.58
[9, 60] loss: 0.459
[9, 120] loss: 0.462
[9, 180] loss: 0.468
[9, 240] loss: 0.454
[9, 300] loss: 0.463
[9, 360] loss: 0.456
Epoch: 9 -> Loss: 0.346526294947
Epoch: 9 -> Test Accuracy: 78.99
[10, 60] loss: 0.461
[10, 120] loss: 0.458
[10, 180] loss: 0.457
[10, 240] loss: 0.460
[10, 300] loss: 0.452
[10, 360] loss: 0.464
Epoch: 10 -> Loss: 0.371485412121
Epoch: 10 -> Test Accuracy: 79.56
[11, 60] loss: 0.456
[11, 120] loss: 0.455
[11, 180] loss: 0.445
[11, 240] loss: 0.463
[11, 300] loss: 0.462
[11, 360] loss: 0.458
Epoch: 11 -> Loss: 0.404909223318
Epoch: 11 -> Test Accuracy: 78.25
[12, 60] loss: 0.445
[12, 120] loss: 0.431
[12, 180] loss: 0.461
[12, 240] loss: 0.469
[12, 300] loss: 0.449
[12, 360] loss: 0.459
Epoch: 12 -> Loss: 0.296258121729
Epoch: 12 -> Test Accuracy: 78.54
[13, 60] loss: 0.456


[49, 240] loss: 0.292
[49, 300] loss: 0.300
[49, 360] loss: 0.298
Epoch: 49 -> Loss: 0.386801183224
Epoch: 49 -> Test Accuracy: 81.21
[50, 60] loss: 0.272
[50, 120] loss: 0.293
[50, 180] loss: 0.292
[50, 240] loss: 0.283
[50, 300] loss: 0.301
[50, 360] loss: 0.291
Epoch: 50 -> Loss: 0.23200365901
Epoch: 50 -> Test Accuracy: 81.19
[51, 60] loss: 0.274
[51, 120] loss: 0.291
[51, 180] loss: 0.286
[51, 240] loss: 0.296
[51, 300] loss: 0.292
[51, 360] loss: 0.287
Epoch: 51 -> Loss: 0.242871567607
Epoch: 51 -> Test Accuracy: 80.92
[52, 60] loss: 0.273
[52, 120] loss: 0.280
[52, 180] loss: 0.297
[52, 240] loss: 0.278
[52, 300] loss: 0.287
[52, 360] loss: 0.302
Epoch: 52 -> Loss: 0.286178827286
Epoch: 52 -> Test Accuracy: 80.63
[53, 60] loss: 0.278
[53, 120] loss: 0.274
[53, 180] loss: 0.295
[53, 240] loss: 0.283
[53, 300] loss: 0.305
[53, 360] loss: 0.307
Epoch: 53 -> Loss: 0.316494882107
Epoch: 53 -> Test Accuracy: 81.29
[54, 60] loss: 0.265
[54, 120] loss: 0.271
[54, 180] loss: 0.286
[54, 2

[90, 360] loss: 0.146
Epoch: 90 -> Loss: 0.12051513046
Epoch: 90 -> Test Accuracy: 82.93
[91, 60] loss: 0.137
[91, 120] loss: 0.147
[91, 180] loss: 0.136
[91, 240] loss: 0.146
[91, 300] loss: 0.142
[91, 360] loss: 0.139
Epoch: 91 -> Loss: 0.120810866356
Epoch: 91 -> Test Accuracy: 82.75
[92, 60] loss: 0.136
[92, 120] loss: 0.145
[92, 180] loss: 0.139
[92, 240] loss: 0.148
[92, 300] loss: 0.137
[92, 360] loss: 0.142
Epoch: 92 -> Loss: 0.225755020976
Epoch: 92 -> Test Accuracy: 82.93
[93, 60] loss: 0.138
[93, 120] loss: 0.133
[93, 180] loss: 0.140
[93, 240] loss: 0.149
[93, 300] loss: 0.138
[93, 360] loss: 0.141
Epoch: 93 -> Loss: 0.134782940149
Epoch: 93 -> Test Accuracy: 82.84
[94, 60] loss: 0.135
[94, 120] loss: 0.136
[94, 180] loss: 0.134
[94, 240] loss: 0.146
[94, 300] loss: 0.136
[94, 360] loss: 0.143
Epoch: 94 -> Loss: 0.169268041849
Epoch: 94 -> Test Accuracy: 82.68
[95, 60] loss: 0.131
[95, 120] loss: 0.143
[95, 180] loss: 0.138
[95, 240] loss: 0.142
[95, 300] loss: 0.140
[95, 3

[32, 180] loss: 1.521
[32, 240] loss: 1.473
[32, 300] loss: 1.516
[32, 360] loss: 1.488
Epoch: 32 -> Loss: 1.553809762
Epoch: 32 -> Test Accuracy: 40.61
[33, 60] loss: 1.472
[33, 120] loss: 1.511
[33, 180] loss: 1.495
[33, 240] loss: 1.501
[33, 300] loss: 1.509
[33, 360] loss: 1.480
Epoch: 33 -> Loss: 1.49745738506
Epoch: 33 -> Test Accuracy: 41.26
[34, 60] loss: 1.484
[34, 120] loss: 1.496
[34, 180] loss: 1.499
[34, 240] loss: 1.488
[34, 300] loss: 1.487
[34, 360] loss: 1.514
Epoch: 34 -> Loss: 1.6510477066
Epoch: 34 -> Test Accuracy: 40.5
[35, 60] loss: 1.478
[35, 120] loss: 1.504
[35, 180] loss: 1.488
[35, 240] loss: 1.498
[35, 300] loss: 1.503
[35, 360] loss: 1.496
Epoch: 35 -> Loss: 1.45281708241
Epoch: 35 -> Test Accuracy: 41.19
[36, 60] loss: 1.448
[36, 120] loss: 1.396
[36, 180] loss: 1.382
[36, 240] loss: 1.385
[36, 300] loss: 1.392
[36, 360] loss: 1.380
Epoch: 36 -> Loss: 1.28960323334
Epoch: 36 -> Test Accuracy: 45.18
[37, 60] loss: 1.371
[37, 120] loss: 1.356
[37, 180] loss

Epoch: 73 -> Loss: 1.23151230812
Epoch: 73 -> Test Accuracy: 48.6
[74, 60] loss: 1.241
[74, 120] loss: 1.216
[74, 180] loss: 1.226
[74, 240] loss: 1.246
[74, 300] loss: 1.242
[74, 360] loss: 1.242
Epoch: 74 -> Loss: 1.14615464211
Epoch: 74 -> Test Accuracy: 49.37
[75, 60] loss: 1.224
[75, 120] loss: 1.231
[75, 180] loss: 1.231
[75, 240] loss: 1.216
[75, 300] loss: 1.242
[75, 360] loss: 1.251
Epoch: 75 -> Loss: 1.25943422318
Epoch: 75 -> Test Accuracy: 49.04
[76, 60] loss: 1.234
[76, 120] loss: 1.241
[76, 180] loss: 1.230
[76, 240] loss: 1.214
[76, 300] loss: 1.224
[76, 360] loss: 1.226
Epoch: 76 -> Loss: 1.12891149521
Epoch: 76 -> Test Accuracy: 49.07
[77, 60] loss: 1.246
[77, 120] loss: 1.237
[77, 180] loss: 1.204
[77, 240] loss: 1.227
[77, 300] loss: 1.248
[77, 360] loss: 1.223
Epoch: 77 -> Loss: 1.43607461452
Epoch: 77 -> Test Accuracy: 48.54
[78, 60] loss: 1.234
[78, 120] loss: 1.213
[78, 180] loss: 1.234
[78, 240] loss: 1.227
[78, 300] loss: 1.229
[78, 360] loss: 1.221
Epoch: 78 -

In [22]:
# save variables
fm.save_variable([rot_block4_loss_log, rot_block4_test_accuracy_log, 
                  block4_loss_log, block4_test_accuracy_log, 
                  conv_block4_loss_log, conv_block4_test_accuracy_log], "4_block_net")

In [23]:
# rename files
fm.add_block_to_name(4, [100, 200])

## 5 Block RotNet

In [27]:
# initialize network
net_block5 = RN.RotNet(num_classes=4, num_conv_block=5, add_avg_pool=False)

In [28]:
# train network
rot_block5_loss_log, _, rot_block5_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block5, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

[1, 60] loss: 1.149
[1, 120] loss: 1.019
[1, 180] loss: 0.948
[1, 240] loss: 0.885
[1, 300] loss: 0.867
[1, 360] loss: 0.832
Epoch: 1 -> Loss: 0.881160914898
Epoch: 1 -> Test Accuracy: 67.94
[2, 60] loss: 0.787
[2, 120] loss: 0.742
[2, 180] loss: 0.718
[2, 240] loss: 0.717
[2, 300] loss: 0.681
[2, 360] loss: 0.668
Epoch: 2 -> Loss: 0.548422813416
Epoch: 2 -> Test Accuracy: 74.82
[3, 60] loss: 0.640
[3, 120] loss: 0.622
[3, 180] loss: 0.628
[3, 240] loss: 0.603
[3, 300] loss: 0.602
[3, 360] loss: 0.575
Epoch: 3 -> Loss: 0.407884776592
Epoch: 3 -> Test Accuracy: 77.97
[4, 60] loss: 0.547
[4, 120] loss: 0.557
[4, 180] loss: 0.546
[4, 240] loss: 0.570
[4, 300] loss: 0.545
[4, 360] loss: 0.537
Epoch: 4 -> Loss: 0.592630028725
Epoch: 4 -> Test Accuracy: 79.935
[5, 60] loss: 0.518
[5, 120] loss: 0.503
[5, 180] loss: 0.511
[5, 240] loss: 0.495
[5, 300] loss: 0.518
[5, 360] loss: 0.493
Epoch: 5 -> Loss: 0.569274783134
Epoch: 5 -> Test Accuracy: 80.275
[6, 60] loss: 0.470
[6, 120] loss: 0.492
[6

[42, 300] loss: 0.314
[42, 360] loss: 0.312
Epoch: 42 -> Loss: 0.253769606352
Epoch: 42 -> Test Accuracy: 87.0525
[43, 60] loss: 0.312
[43, 120] loss: 0.304
[43, 180] loss: 0.323
[43, 240] loss: 0.311
[43, 300] loss: 0.316
[43, 360] loss: 0.307
Epoch: 43 -> Loss: 0.563426613808
Epoch: 43 -> Test Accuracy: 85.84
[44, 60] loss: 0.308
[44, 120] loss: 0.311
[44, 180] loss: 0.314
[44, 240] loss: 0.314
[44, 300] loss: 0.314
[44, 360] loss: 0.316
Epoch: 44 -> Loss: 0.197260409594
Epoch: 44 -> Test Accuracy: 85.445
[45, 60] loss: 0.301
[45, 120] loss: 0.307
[45, 180] loss: 0.311
[45, 240] loss: 0.306
[45, 300] loss: 0.329
[45, 360] loss: 0.307
Epoch: 45 -> Loss: 0.355705320835
Epoch: 45 -> Test Accuracy: 86.38
[46, 60] loss: 0.307
[46, 120] loss: 0.321
[46, 180] loss: 0.298
[46, 240] loss: 0.311
[46, 300] loss: 0.313
[46, 360] loss: 0.318
Epoch: 46 -> Loss: 0.340223789215
Epoch: 46 -> Test Accuracy: 86.5175
[47, 60] loss: 0.292
[47, 120] loss: 0.294
[47, 180] loss: 0.313
[47, 240] loss: 0.324


[83, 300] loss: 0.144
[83, 360] loss: 0.142
Epoch: 83 -> Loss: 0.18048402667
Epoch: 83 -> Test Accuracy: 90.06
[84, 60] loss: 0.140
[84, 120] loss: 0.142
[84, 180] loss: 0.147
[84, 240] loss: 0.153
[84, 300] loss: 0.153
[84, 360] loss: 0.148
Epoch: 84 -> Loss: 0.216561958194
Epoch: 84 -> Test Accuracy: 90.1975
[85, 60] loss: 0.144
[85, 120] loss: 0.150
[85, 180] loss: 0.148
[85, 240] loss: 0.147
[85, 300] loss: 0.151
[85, 360] loss: 0.148
Epoch: 85 -> Loss: 0.126949474216
Epoch: 85 -> Test Accuracy: 90.1625
[86, 60] loss: 0.131
[86, 120] loss: 0.144
[86, 180] loss: 0.150
[86, 240] loss: 0.149
[86, 300] loss: 0.146
[86, 360] loss: 0.153
Epoch: 86 -> Loss: 0.150083363056
Epoch: 86 -> Test Accuracy: 90.045
[87, 60] loss: 0.142
[87, 120] loss: 0.133
[87, 180] loss: 0.146
[87, 240] loss: 0.146
[87, 300] loss: 0.148
[87, 360] loss: 0.153
Epoch: 87 -> Loss: 0.199031338096
Epoch: 87 -> Test Accuracy: 90.345
[88, 60] loss: 0.133
[88, 120] loss: 0.137
[88, 180] loss: 0.141
[88, 240] loss: 0.153


[123, 300] loss: 0.049
[123, 360] loss: 0.047
Epoch: 123 -> Loss: 0.0200503431261
Epoch: 123 -> Test Accuracy: 91.94
[124, 60] loss: 0.041
[124, 120] loss: 0.042
[124, 180] loss: 0.042
[124, 240] loss: 0.044
[124, 300] loss: 0.044
[124, 360] loss: 0.044
Epoch: 124 -> Loss: 0.0708405822515
Epoch: 124 -> Test Accuracy: 91.84
[125, 60] loss: 0.040
[125, 120] loss: 0.040
[125, 180] loss: 0.039
[125, 240] loss: 0.042
[125, 300] loss: 0.041
[125, 360] loss: 0.042
Epoch: 125 -> Loss: 0.0240495614707
Epoch: 125 -> Test Accuracy: 92.16
[126, 60] loss: 0.039
[126, 120] loss: 0.036
[126, 180] loss: 0.038
[126, 240] loss: 0.041
[126, 300] loss: 0.036
[126, 360] loss: 0.037
Epoch: 126 -> Loss: 0.0288580060005
Epoch: 126 -> Test Accuracy: 91.96
[127, 60] loss: 0.036
[127, 120] loss: 0.039
[127, 180] loss: 0.035
[127, 240] loss: 0.040
[127, 300] loss: 0.038
[127, 360] loss: 0.038
Epoch: 127 -> Loss: 0.0375020503998
Epoch: 127 -> Test Accuracy: 91.9625
[128, 60] loss: 0.032
[128, 120] loss: 0.036
[128

Epoch: 162 -> Loss: 0.0203562006354
Epoch: 162 -> Test Accuracy: 92.095
[163, 60] loss: 0.012
[163, 120] loss: 0.010
[163, 180] loss: 0.010
[163, 240] loss: 0.011
[163, 300] loss: 0.010
[163, 360] loss: 0.012
Epoch: 163 -> Loss: 0.0027140781749
Epoch: 163 -> Test Accuracy: 92.1275
[164, 60] loss: 0.009
[164, 120] loss: 0.009
[164, 180] loss: 0.012
[164, 240] loss: 0.009
[164, 300] loss: 0.009
[164, 360] loss: 0.010
Epoch: 164 -> Loss: 0.00260318745859
Epoch: 164 -> Test Accuracy: 92.12
[165, 60] loss: 0.009
[165, 120] loss: 0.009
[165, 180] loss: 0.009
[165, 240] loss: 0.009
[165, 300] loss: 0.008
[165, 360] loss: 0.010
Epoch: 165 -> Loss: 0.00682274857536
Epoch: 165 -> Test Accuracy: 92.11
[166, 60] loss: 0.009
[166, 120] loss: 0.008
[166, 180] loss: 0.009
[166, 240] loss: 0.009
[166, 300] loss: 0.007
[166, 360] loss: 0.010
Epoch: 166 -> Loss: 0.00325839081779
Epoch: 166 -> Test Accuracy: 92.1675
[167, 60] loss: 0.009
[167, 120] loss: 0.007
[167, 180] loss: 0.007
[167, 240] loss: 0.00

In [29]:
# train NonLinearClassifiers on feature map of net_3block
block5_loss_log, _, block5_test_accuracy_log, _, _ = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 100], 0.9, 5e-4, net_block5, criterion, trainloader, None, testloader) 

[1, 60] loss: 2.203
[1, 120] loss: 1.267
[1, 180] loss: 1.140
[1, 240] loss: 1.080
[1, 300] loss: 1.048
[1, 360] loss: 0.999
Epoch: 1 -> Loss: 0.934138476849
Epoch: 1 -> Test Accuracy: 67.18
[2, 60] loss: 0.946
[2, 120] loss: 0.918
[2, 180] loss: 0.906
[2, 240] loss: 0.909
[2, 300] loss: 0.889
[2, 360] loss: 0.874
Epoch: 2 -> Loss: 0.738484025002
Epoch: 2 -> Test Accuracy: 70.94
[3, 60] loss: 0.828
[3, 120] loss: 0.838
[3, 180] loss: 0.823
[3, 240] loss: 0.814
[3, 300] loss: 0.801
[3, 360] loss: 0.804
Epoch: 3 -> Loss: 0.982511878014
Epoch: 3 -> Test Accuracy: 72.24
[4, 60] loss: 0.790
[4, 120] loss: 0.764
[4, 180] loss: 0.790
[4, 240] loss: 0.771
[4, 300] loss: 0.746
[4, 360] loss: 0.756
Epoch: 4 -> Loss: 0.804670989513
Epoch: 4 -> Test Accuracy: 73.25
[5, 60] loss: 0.745
[5, 120] loss: 0.733
[5, 180] loss: 0.741
[5, 240] loss: 0.734
[5, 300] loss: 0.732
[5, 360] loss: 0.716
Epoch: 5 -> Loss: 0.571606516838
Epoch: 5 -> Test Accuracy: 75.17
[6, 60] loss: 0.707
[6, 120] loss: 0.707
[6, 

Epoch: 42 -> Loss: 0.318571090698
Epoch: 42 -> Test Accuracy: 82.16
[43, 60] loss: 0.327
[43, 120] loss: 0.311
[43, 180] loss: 0.309
[43, 240] loss: 0.317
[43, 300] loss: 0.322
[43, 360] loss: 0.330
Epoch: 43 -> Loss: 0.446107953787
Epoch: 43 -> Test Accuracy: 82.4
[44, 60] loss: 0.313
[44, 120] loss: 0.319
[44, 180] loss: 0.314
[44, 240] loss: 0.322
[44, 300] loss: 0.300
[44, 360] loss: 0.313
Epoch: 44 -> Loss: 0.303883850574
Epoch: 44 -> Test Accuracy: 82.91
[45, 60] loss: 0.307
[45, 120] loss: 0.305
[45, 180] loss: 0.308
[45, 240] loss: 0.300
[45, 300] loss: 0.307
[45, 360] loss: 0.304
Epoch: 45 -> Loss: 0.415184259415
Epoch: 45 -> Test Accuracy: 82.23
[46, 60] loss: 0.297
[46, 120] loss: 0.296
[46, 180] loss: 0.298
[46, 240] loss: 0.286
[46, 300] loss: 0.296
[46, 360] loss: 0.293
Epoch: 46 -> Loss: 0.295275509357
Epoch: 46 -> Test Accuracy: 82.62
[47, 60] loss: 0.289
[47, 120] loss: 0.298
[47, 180] loss: 0.286
[47, 240] loss: 0.298
[47, 300] loss: 0.309
[47, 360] loss: 0.287
Epoch:

[84, 60] loss: 0.251
[84, 120] loss: 0.256
[84, 180] loss: 0.244
[84, 240] loss: 0.251
[84, 300] loss: 0.247
[84, 360] loss: 0.248
Epoch: 84 -> Loss: 0.289580643177
Epoch: 84 -> Test Accuracy: 83.07
[85, 60] loss: 0.244
[85, 120] loss: 0.251
[85, 180] loss: 0.248
[85, 240] loss: 0.252
[85, 300] loss: 0.249
[85, 360] loss: 0.251
Epoch: 85 -> Loss: 0.266908824444
Epoch: 85 -> Test Accuracy: 82.99
[86, 60] loss: 0.240
[86, 120] loss: 0.239
[86, 180] loss: 0.242
[86, 240] loss: 0.258
[86, 300] loss: 0.237
[86, 360] loss: 0.242
Epoch: 86 -> Loss: 0.249841243029
Epoch: 86 -> Test Accuracy: 83.05
[87, 60] loss: 0.262
[87, 120] loss: 0.243
[87, 180] loss: 0.245
[87, 240] loss: 0.242
[87, 300] loss: 0.247
[87, 360] loss: 0.250
Epoch: 87 -> Loss: 0.377417176962
Epoch: 87 -> Test Accuracy: 83.1
[88, 60] loss: 0.237
[88, 120] loss: 0.249
[88, 180] loss: 0.253
[88, 240] loss: 0.252
[88, 300] loss: 0.236
[88, 360] loss: 0.251
Epoch: 88 -> Loss: 0.178559452295
Epoch: 88 -> Test Accuracy: 82.99
[89, 6

[25, 360] loss: 0.240
Epoch: 25 -> Loss: 0.278952866793
Epoch: 25 -> Test Accuracy: 85.61
[26, 60] loss: 0.223
[26, 120] loss: 0.222
[26, 180] loss: 0.216
[26, 240] loss: 0.216
[26, 300] loss: 0.217
[26, 360] loss: 0.222
Epoch: 26 -> Loss: 0.139011666179
Epoch: 26 -> Test Accuracy: 85.63
[27, 60] loss: 0.224
[27, 120] loss: 0.208
[27, 180] loss: 0.213
[27, 240] loss: 0.224
[27, 300] loss: 0.228
[27, 360] loss: 0.217
Epoch: 27 -> Loss: 0.278864204884
Epoch: 27 -> Test Accuracy: 85.71
[28, 60] loss: 0.206
[28, 120] loss: 0.200
[28, 180] loss: 0.214
[28, 240] loss: 0.214
[28, 300] loss: 0.236
[28, 360] loss: 0.212
Epoch: 28 -> Loss: 0.26783233881
Epoch: 28 -> Test Accuracy: 85.67
[29, 60] loss: 0.203
[29, 120] loss: 0.204
[29, 180] loss: 0.202
[29, 240] loss: 0.203
[29, 300] loss: 0.229
[29, 360] loss: 0.218
Epoch: 29 -> Loss: 0.166415497661
Epoch: 29 -> Test Accuracy: 85.47
[30, 60] loss: 0.203
[30, 120] loss: 0.201
[30, 180] loss: 0.205
[30, 240] loss: 0.203
[30, 300] loss: 0.209
[30, 3

Epoch: 66 -> Test Accuracy: 86.68
[67, 60] loss: 0.095
[67, 120] loss: 0.091
[67, 180] loss: 0.098
[67, 240] loss: 0.105
[67, 300] loss: 0.097
[67, 360] loss: 0.102
Epoch: 67 -> Loss: 0.224867194891
Epoch: 67 -> Test Accuracy: 86.77
[68, 60] loss: 0.088
[68, 120] loss: 0.095
[68, 180] loss: 0.094
[68, 240] loss: 0.097
[68, 300] loss: 0.099
[68, 360] loss: 0.091
Epoch: 68 -> Loss: 0.177250146866
Epoch: 68 -> Test Accuracy: 86.81
[69, 60] loss: 0.092
[69, 120] loss: 0.104
[69, 180] loss: 0.099
[69, 240] loss: 0.097
[69, 300] loss: 0.096
[69, 360] loss: 0.096
Epoch: 69 -> Loss: 0.110779598355
Epoch: 69 -> Test Accuracy: 86.73
[70, 60] loss: 0.100
[70, 120] loss: 0.102
[70, 180] loss: 0.094
[70, 240] loss: 0.088
[70, 300] loss: 0.093
[70, 360] loss: 0.099
Epoch: 70 -> Loss: 0.0637176558375
Epoch: 70 -> Test Accuracy: 86.73
[71, 60] loss: 0.092
[71, 120] loss: 0.092
[71, 180] loss: 0.095
[71, 240] loss: 0.089
[71, 300] loss: 0.089
[71, 360] loss: 0.089
Epoch: 71 -> Loss: 0.0852275788784
Epo

...

[32, 300] loss: 0.744
[32, 360] loss: 0.786
Epoch: 32 -> Loss: 0.690040707588
Epoch: 32 -> Test Accuracy: 66.67
[33, 60] loss: 0.743
[33, 120] loss: 0.766
[33, 180] loss: 0.795
[33, 240] loss: 0.792
[33, 300] loss: 0.764
[33, 360] loss: 0.770
Epoch: 33 -> Loss: 0.762346923351
Epoch: 33 -> Test Accuracy: 65.95
[34, 60] loss: 0.767
[34, 120] loss: 0.775
[34, 180] loss: 0.757
[34, 240] loss: 0.787
[34, 300] loss: 0.761
[34, 360] loss: 0.792
Epoch: 34 -> Loss: 0.779279351234
Epoch: 34 -> Test Accuracy: 66.19
[35, 60] loss: 0.756
[35, 120] loss: 0.760
[35, 180] loss: 0.769
[35, 240] loss: 0.775
[35, 300] loss: 0.750
[35, 360] loss: 0.779
Epoch: 35 -> Loss: 0.849234938622
Epoch: 35 -> Test Accuracy: 66.46
[36, 60] loss: 0.766
[36, 120] loss: 0.775
[36, 180] loss: 0.779
[36, 240] loss: 0.763
[36, 300] loss: 0.766
[36, 360] loss: 0.771
Epoch: 36 -> Loss: 0.688422560692
Epoch: 36 -> Test Accuracy: 66.25
[37, 60] loss: 0.763
[37, 120] loss: 0.778
[37, 180] loss: 0.780
[37, 240] loss: 0.772
[37, 

Epoch: 73 -> Loss: 0.700736165047
Epoch: 73 -> Test Accuracy: 69.54
[74, 60] loss: 0.647
[74, 120] loss: 0.634
[74, 180] loss: 0.667
[74, 240] loss: 0.639
[74, 300] loss: 0.641
[74, 360] loss: 0.618
Epoch: 74 -> Loss: 0.669369578362
Epoch: 74 -> Test Accuracy: 69.71
[75, 60] loss: 0.645
[75, 120] loss: 0.641
[75, 180] loss: 0.629
[75, 240] loss: 0.642
[75, 300] loss: 0.644
[75, 360] loss: 0.644
Epoch: 75 -> Loss: 0.803560137749
Epoch: 75 -> Test Accuracy: 69.62
[76, 60] loss: 0.642
[76, 120] loss: 0.633
[76, 180] loss: 0.640
[76, 240] loss: 0.647
[76, 300] loss: 0.662
[76, 360] loss: 0.637
Epoch: 76 -> Loss: 0.694324374199
Epoch: 76 -> Test Accuracy: 69.74
[77, 60] loss: 0.633
[77, 120] loss: 0.630
[77, 180] loss: 0.655
[77, 240] loss: 0.650
[77, 300] loss: 0.641
[77, 360] loss: 0.630
Epoch: 77 -> Loss: 0.598105728626
Epoch: 77 -> Test Accuracy: 69.8
[78, 60] loss: 0.644
[78, 120] loss: 0.619
[78, 180] loss: 0.648
[78, 240] loss: 0.615
[78, 300] loss: 0.636
[78, 360] loss: 0.645
Epoch:

[15, 180] loss: 1.806
[15, 240] loss: 1.802
[15, 300] loss: 1.831
[15, 360] loss: 1.825
Epoch: 15 -> Loss: 1.87135100365
Epoch: 15 -> Test Accuracy: 31.19
[16, 60] loss: 1.830
[16, 120] loss: 1.826
[16, 180] loss: 1.823
[16, 240] loss: 1.812
[16, 300] loss: 1.828
[16, 360] loss: 1.823
Epoch: 16 -> Loss: 1.66989779472
Epoch: 16 -> Test Accuracy: 31.25
[17, 60] loss: 1.809
[17, 120] loss: 1.794
[17, 180] loss: 1.814
[17, 240] loss: 1.815
[17, 300] loss: 1.826
[17, 360] loss: 1.788
Epoch: 17 -> Loss: 1.80468523502
Epoch: 17 -> Test Accuracy: 30.83
[18, 60] loss: 1.824
[18, 120] loss: 1.809
[18, 180] loss: 1.807
[18, 240] loss: 1.823
[18, 300] loss: 1.825
[18, 360] loss: 1.813
Epoch: 18 -> Loss: 1.84267365932
Epoch: 18 -> Test Accuracy: 31.14
[19, 60] loss: 1.816
[19, 120] loss: 1.798
[19, 180] loss: 1.813
[19, 240] loss: 1.817
[19, 300] loss: 1.817
[19, 360] loss: 1.828
Epoch: 19 -> Loss: 1.64079380035
Epoch: 19 -> Test Accuracy: 31.6
[20, 60] loss: 1.813
[20, 120] loss: 1.827
[20, 180] l

Epoch: 56 -> Test Accuracy: 36.89
[57, 60] loss: 1.607
[57, 120] loss: 1.616
[57, 180] loss: 1.618
[57, 240] loss: 1.590
[57, 300] loss: 1.621
[57, 360] loss: 1.615
Epoch: 57 -> Loss: 1.46420919895
Epoch: 57 -> Test Accuracy: 36.88
[58, 60] loss: 1.608
[58, 120] loss: 1.610
[58, 180] loss: 1.607
[58, 240] loss: 1.610
[58, 300] loss: 1.616
[58, 360] loss: 1.608
Epoch: 58 -> Loss: 1.75742685795
Epoch: 58 -> Test Accuracy: 36.81
[59, 60] loss: 1.598
[59, 120] loss: 1.585
[59, 180] loss: 1.594
[59, 240] loss: 1.612
[59, 300] loss: 1.597
[59, 360] loss: 1.605
Epoch: 59 -> Loss: 1.52518963814
Epoch: 59 -> Test Accuracy: 37.04
[60, 60] loss: 1.610
[60, 120] loss: 1.599
[60, 180] loss: 1.614
[60, 240] loss: 1.600
[60, 300] loss: 1.598
[60, 360] loss: 1.599
Epoch: 60 -> Loss: 1.6344563961
Epoch: 60 -> Test Accuracy: 36.89
[61, 60] loss: 1.605
[61, 120] loss: 1.591
[61, 180] loss: 1.587
[61, 240] loss: 1.618
[61, 300] loss: 1.603
[61, 360] loss: 1.604
Epoch: 61 -> Loss: 1.56204903126
Epoch: 61 -

[98, 240] loss: 1.592
[98, 300] loss: 1.579
[98, 360] loss: 1.588
Epoch: 98 -> Loss: 1.7388446331
Epoch: 98 -> Test Accuracy: 37.05
[99, 60] loss: 1.581
[99, 120] loss: 1.591
[99, 180] loss: 1.606
[99, 240] loss: 1.593
[99, 300] loss: 1.597
[99, 360] loss: 1.579
Epoch: 99 -> Loss: 1.52238571644
Epoch: 99 -> Test Accuracy: 36.73
[100, 60] loss: 1.608
[100, 120] loss: 1.571
[100, 180] loss: 1.580
[100, 240] loss: 1.594
[100, 300] loss: 1.588
[100, 360] loss: 1.569
Epoch: 100 -> Loss: 1.73447918892
Epoch: 100 -> Test Accuracy: 36.9
Finished Training


In [30]:
# train ConvClassifiers on feature map of net_3block
conv_block5_loss_log, _, conv_block5_test_accuracy_log, _, _ = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], 
    [35, 70, 85, 100], 0.9, 5e-4, net_block5, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

[1, 60] loss: 1.415
[1, 120] loss: 1.049
[1, 180] loss: 0.962
[1, 240] loss: 0.913
[1, 300] loss: 0.846
[1, 360] loss: 0.820
Epoch: 1 -> Loss: 0.900788605213
Epoch: 1 -> Test Accuracy: 69.86
[2, 60] loss: 0.756
[2, 120] loss: 0.748
[2, 180] loss: 0.732
[2, 240] loss: 0.701
[2, 300] loss: 0.706
[2, 360] loss: 0.709
Epoch: 2 -> Loss: 0.587452173233
Epoch: 2 -> Test Accuracy: 74.13
[3, 60] loss: 0.655
[3, 120] loss: 0.656
[3, 180] loss: 0.636
[3, 240] loss: 0.640
[3, 300] loss: 0.628
[3, 360] loss: 0.605
Epoch: 3 -> Loss: 0.866232037544
Epoch: 3 -> Test Accuracy: 74.97
[4, 60] loss: 0.619
[4, 120] loss: 0.593
[4, 180] loss: 0.610
[4, 240] loss: 0.574
[4, 300] loss: 0.585
[4, 360] loss: 0.591
Epoch: 4 -> Loss: 0.621171534061
Epoch: 4 -> Test Accuracy: 76.79
[5, 60] loss: 0.560
[5, 120] loss: 0.555
[5, 180] loss: 0.566
[5, 240] loss: 0.574
[5, 300] loss: 0.555
[5, 360] loss: 0.582
Epoch: 5 -> Loss: 0.68909573555
Epoch: 5 -> Test Accuracy: 76.13
[6, 60] loss: 0.532
[6, 120] loss: 0.536
[6, 1

Epoch: 42 -> Loss: 0.315601766109
Epoch: 42 -> Test Accuracy: 83.94
[43, 60] loss: 0.222
[43, 120] loss: 0.227
[43, 180] loss: 0.230
[43, 240] loss: 0.232
[43, 300] loss: 0.238
[43, 360] loss: 0.238
Epoch: 43 -> Loss: 0.196926355362
Epoch: 43 -> Test Accuracy: 84.69
[44, 60] loss: 0.210
[44, 120] loss: 0.224
[44, 180] loss: 0.236
[44, 240] loss: 0.240
[44, 300] loss: 0.233
[44, 360] loss: 0.247
Epoch: 44 -> Loss: 0.221259504557
Epoch: 44 -> Test Accuracy: 85.13
[45, 60] loss: 0.219
[45, 120] loss: 0.227
[45, 180] loss: 0.232
[45, 240] loss: 0.227
[45, 300] loss: 0.247
[45, 360] loss: 0.240
Epoch: 45 -> Loss: 0.303429782391
Epoch: 45 -> Test Accuracy: 83.88
[46, 60] loss: 0.221
[46, 120] loss: 0.230
[46, 180] loss: 0.218
[46, 240] loss: 0.235
[46, 300] loss: 0.236
[46, 360] loss: 0.237
Epoch: 46 -> Loss: 0.325609445572
Epoch: 46 -> Test Accuracy: 84.49
[47, 60] loss: 0.219
[47, 120] loss: 0.227
[47, 180] loss: 0.225
[47, 240] loss: 0.227
[47, 300] loss: 0.235
[47, 360] loss: 0.243
Epoch

[84, 60] loss: 0.096
[84, 120] loss: 0.096
[84, 180] loss: 0.105
[84, 240] loss: 0.107
[84, 300] loss: 0.109
[84, 360] loss: 0.108
Epoch: 84 -> Loss: 0.117718979716
Epoch: 84 -> Test Accuracy: 85.77
[85, 60] loss: 0.101
[85, 120] loss: 0.096
[85, 180] loss: 0.104
[85, 240] loss: 0.102
[85, 300] loss: 0.110
[85, 360] loss: 0.104
Epoch: 85 -> Loss: 0.123285248876
Epoch: 85 -> Test Accuracy: 85.89
[86, 60] loss: 0.093
[86, 120] loss: 0.095
[86, 180] loss: 0.091
[86, 240] loss: 0.094
[86, 300] loss: 0.089
[86, 360] loss: 0.094
Epoch: 86 -> Loss: 0.120373107493
Epoch: 86 -> Test Accuracy: 85.96
[87, 60] loss: 0.085
[87, 120] loss: 0.087
[87, 180] loss: 0.086
[87, 240] loss: 0.092
[87, 300] loss: 0.088
[87, 360] loss: 0.088
Epoch: 87 -> Loss: 0.0820851475
Epoch: 87 -> Test Accuracy: 86.16
[88, 60] loss: 0.088
[88, 120] loss: 0.088
[88, 180] loss: 0.087
[88, 240] loss: 0.087
[88, 300] loss: 0.089
[88, 360] loss: 0.085
Epoch: 88 -> Loss: 0.104723110795
Epoch: 88 -> Test Accuracy: 86.13
[89, 60

[25, 300] loss: 0.276
[25, 360] loss: 0.279
Epoch: 25 -> Loss: 0.307028323412
Epoch: 25 -> Test Accuracy: 84.01
[26, 60] loss: 0.250
[26, 120] loss: 0.260
[26, 180] loss: 0.285
[26, 240] loss: 0.265
[26, 300] loss: 0.286
[26, 360] loss: 0.282
Epoch: 26 -> Loss: 0.22989718616
Epoch: 26 -> Test Accuracy: 85.62
[27, 60] loss: 0.259
[27, 120] loss: 0.252
[27, 180] loss: 0.266
[27, 240] loss: 0.285
[27, 300] loss: 0.280
[27, 360] loss: 0.288
Epoch: 27 -> Loss: 0.347602188587
Epoch: 27 -> Test Accuracy: 84.25
[28, 60] loss: 0.260
[28, 120] loss: 0.257
[28, 180] loss: 0.268
[28, 240] loss: 0.275
[28, 300] loss: 0.276
[28, 360] loss: 0.277
Epoch: 28 -> Loss: 0.302180826664
Epoch: 28 -> Test Accuracy: 85.22
[29, 60] loss: 0.242
[29, 120] loss: 0.256
[29, 180] loss: 0.266
[29, 240] loss: 0.283
[29, 300] loss: 0.271
[29, 360] loss: 0.256
Epoch: 29 -> Loss: 0.359811782837
Epoch: 29 -> Test Accuracy: 84.81
[30, 60] loss: 0.252
[30, 120] loss: 0.238
[30, 180] loss: 0.268
[30, 240] loss: 0.257
[30, 3

Epoch: 66 -> Loss: 0.0944371372461
Epoch: 66 -> Test Accuracy: 86.86
[67, 60] loss: 0.097
[67, 120] loss: 0.095
[67, 180] loss: 0.098
[67, 240] loss: 0.099
[67, 300] loss: 0.112
[67, 360] loss: 0.116
Epoch: 67 -> Loss: 0.0530914664268
Epoch: 67 -> Test Accuracy: 86.55
[68, 60] loss: 0.094
[68, 120] loss: 0.094
[68, 180] loss: 0.099
[68, 240] loss: 0.104
[68, 300] loss: 0.107
[68, 360] loss: 0.116
Epoch: 68 -> Loss: 0.143045663834
Epoch: 68 -> Test Accuracy: 86.5
[69, 60] loss: 0.096
[69, 120] loss: 0.098
[69, 180] loss: 0.103
[69, 240] loss: 0.097
[69, 300] loss: 0.107
[69, 360] loss: 0.114
Epoch: 69 -> Loss: 0.0928893759847
Epoch: 69 -> Test Accuracy: 86.9
[70, 60] loss: 0.093
[70, 120] loss: 0.087
[70, 180] loss: 0.096
[70, 240] loss: 0.104
[70, 300] loss: 0.100
[70, 360] loss: 0.115
Epoch: 70 -> Loss: 0.172365143895
Epoch: 70 -> Test Accuracy: 86.85
[71, 60] loss: 0.082
[71, 120] loss: 0.070
[71, 180] loss: 0.063
[71, 240] loss: 0.060
[71, 300] loss: 0.058
[71, 360] loss: 0.059
Epoc

...

[32, 60] loss: 0.789
[32, 120] loss: 0.825
[32, 180] loss: 0.800
[32, 240] loss: 0.795
[32, 300] loss: 0.799
[32, 360] loss: 0.816
Epoch: 32 -> Loss: 0.536200046539
Epoch: 32 -> Test Accuracy: 65.8
[33, 60] loss: 0.782
[33, 120] loss: 0.787
[33, 180] loss: 0.818
[33, 240] loss: 0.796
[33, 300] loss: 0.798
[33, 360] loss: 0.813
Epoch: 33 -> Loss: 0.817380428314
Epoch: 33 -> Test Accuracy: 65.58
[34, 60] loss: 0.781
[34, 120] loss: 0.808
[34, 180] loss: 0.817
[34, 240] loss: 0.818
[34, 300] loss: 0.819
[34, 360] loss: 0.795
Epoch: 34 -> Loss: 0.626615107059
Epoch: 34 -> Test Accuracy: 64.77
[35, 60] loss: 0.798
[35, 120] loss: 0.808
[35, 180] loss: 0.811
[35, 240] loss: 0.790
[35, 300] loss: 0.791
[35, 360] loss: 0.796
Epoch: 35 -> Loss: 0.773610949516
Epoch: 35 -> Test Accuracy: 66.33
[36, 60] loss: 0.754
[36, 120] loss: 0.707
[36, 180] loss: 0.700
[36, 240] loss: 0.701
[36, 300] loss: 0.703
[36, 360] loss: 0.703
Epoch: 36 -> Loss: 0.5983902812
Epoch: 36 -> Test Accuracy: 69.6
[37, 60] 

[73, 180] loss: 0.562
[73, 240] loss: 0.580
[73, 300] loss: 0.606
[73, 360] loss: 0.581
Epoch: 73 -> Loss: 0.507665276527
Epoch: 73 -> Test Accuracy: 72.57
[74, 60] loss: 0.572
[74, 120] loss: 0.571
[74, 180] loss: 0.565
[74, 240] loss: 0.582
[74, 300] loss: 0.585
[74, 360] loss: 0.582
Epoch: 74 -> Loss: 0.468992620707
Epoch: 74 -> Test Accuracy: 71.98
[75, 60] loss: 0.568
[75, 120] loss: 0.586
[75, 180] loss: 0.581
[75, 240] loss: 0.591
[75, 300] loss: 0.582
[75, 360] loss: 0.568
Epoch: 75 -> Loss: 0.697091221809
Epoch: 75 -> Test Accuracy: 71.93
[76, 60] loss: 0.579
[76, 120] loss: 0.575
[76, 180] loss: 0.576
[76, 240] loss: 0.575
[76, 300] loss: 0.556
[76, 360] loss: 0.568
Epoch: 76 -> Loss: 0.489638745785
Epoch: 76 -> Test Accuracy: 71.98
[77, 60] loss: 0.568
[77, 120] loss: 0.579
[77, 180] loss: 0.594
[77, 240] loss: 0.558
[77, 300] loss: 0.567
[77, 360] loss: 0.570
Epoch: 77 -> Loss: 0.700900971889
Epoch: 77 -> Test Accuracy: 72.2
[78, 60] loss: 0.581
[78, 120] loss: 0.557
[78, 1

Epoch: 14 -> Test Accuracy: 31.87
[15, 60] loss: 1.754
[15, 120] loss: 1.761
[15, 180] loss: 1.761
[15, 240] loss: 1.753
[15, 300] loss: 1.748
[15, 360] loss: 1.741
Epoch: 15 -> Loss: 1.69936585426
Epoch: 15 -> Test Accuracy: 32.17
[16, 60] loss: 1.735
[16, 120] loss: 1.762
[16, 180] loss: 1.739
[16, 240] loss: 1.763
[16, 300] loss: 1.756
[16, 360] loss: 1.750
Epoch: 16 -> Loss: 1.72331047058
Epoch: 16 -> Test Accuracy: 32.27
[17, 60] loss: 1.731
[17, 120] loss: 1.755
[17, 180] loss: 1.768
[17, 240] loss: 1.740
[17, 300] loss: 1.754
[17, 360] loss: 1.750
Epoch: 17 -> Loss: 1.89817810059
Epoch: 17 -> Test Accuracy: 32.75
[18, 60] loss: 1.747
[18, 120] loss: 1.741
[18, 180] loss: 1.735
[18, 240] loss: 1.752
[18, 300] loss: 1.732
[18, 360] loss: 1.734
Epoch: 18 -> Loss: 1.71640241146
Epoch: 18 -> Test Accuracy: 32.28
[19, 60] loss: 1.741
[19, 120] loss: 1.752
[19, 180] loss: 1.739
[19, 240] loss: 1.740
[19, 300] loss: 1.726
[19, 360] loss: 1.757
Epoch: 19 -> Loss: 1.67088627815
Epoch: 19 

[56, 240] loss: 1.628
[56, 300] loss: 1.614
[56, 360] loss: 1.590
Epoch: 56 -> Loss: 1.59642672539
Epoch: 56 -> Test Accuracy: 36.47
[57, 60] loss: 1.610
[57, 120] loss: 1.610
[57, 180] loss: 1.623
[57, 240] loss: 1.616
[57, 300] loss: 1.611
[57, 360] loss: 1.619
Epoch: 57 -> Loss: 1.41619992256
Epoch: 57 -> Test Accuracy: 36.8
[58, 60] loss: 1.614
[58, 120] loss: 1.598
[58, 180] loss: 1.601
[58, 240] loss: 1.607
[58, 300] loss: 1.609
[58, 360] loss: 1.606
Epoch: 58 -> Loss: 1.74972033501
Epoch: 58 -> Test Accuracy: 37.06
[59, 60] loss: 1.599
[59, 120] loss: 1.613
[59, 180] loss: 1.613
[59, 240] loss: 1.599
[59, 300] loss: 1.600
[59, 360] loss: 1.603
Epoch: 59 -> Loss: 1.46792864799
Epoch: 59 -> Test Accuracy: 37.2
[60, 60] loss: 1.609
[60, 120] loss: 1.597
[60, 180] loss: 1.607
[60, 240] loss: 1.617
[60, 300] loss: 1.613
[60, 360] loss: 1.594
Epoch: 60 -> Loss: 1.64940416813
Epoch: 60 -> Test Accuracy: 37.71
[61, 60] loss: 1.598
[61, 120] loss: 1.605
[61, 180] loss: 1.618
[61, 240] lo

Epoch: 97 -> Test Accuracy: 41.41
[98, 60] loss: 1.489
[98, 120] loss: 1.446
[98, 180] loss: 1.464
[98, 240] loss: 1.485
[98, 300] loss: 1.485
[98, 360] loss: 1.456
Epoch: 98 -> Loss: 1.33560109138
Epoch: 98 -> Test Accuracy: 41.2
[99, 60] loss: 1.463
[99, 120] loss: 1.476
[99, 180] loss: 1.459
[99, 240] loss: 1.502
[99, 300] loss: 1.467
[99, 360] loss: 1.459
Epoch: 99 -> Loss: 1.43488943577
Epoch: 99 -> Test Accuracy: 41.21
[100, 60] loss: 1.484
[100, 120] loss: 1.459
[100, 180] loss: 1.469
[100, 240] loss: 1.468
[100, 300] loss: 1.479
[100, 360] loss: 1.465
Epoch: 100 -> Loss: 1.63368034363
Epoch: 100 -> Test Accuracy: 41.48
Finished Training


In [31]:
# save variables
fm.save_variable([rot_block5_loss_log, rot_block5_test_accuracy_log, 
                  block5_loss_log, block5_test_accuracy_log, 
                  conv_block5_loss_log, conv_block5_test_accuracy_log], "5_block_net")

In [32]:
# rename files
fm.add_block_to_name(5, [100, 200])

# Supervised NIN

Note: In the code of the paper a 3 convolutional block RotNet was used for the classification task.

In [9]:
# initialize networks
net_class = RN.RotNet(num_classes=10, num_conv_block=3, add_avg_pool=False)

In [10]:
# train 3 block RotNet on classification task
class_NIN_loss_log, _, class_NIN_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_class, criterion, trainloader, None, testloader)

[1, 60] loss: 1.751
[1, 120] loss: 1.480
[1, 180] loss: 1.339
[1, 240] loss: 1.254
[1, 300] loss: 1.175
[1, 360] loss: 1.112
Epoch: 1 -> Loss: 0.825036227703
Epoch: 1 -> Test Accuracy: 60.62
[2, 60] loss: 1.038
[2, 120] loss: 0.998
[2, 180] loss: 0.963
[2, 240] loss: 0.930
[2, 300] loss: 0.911
[2, 360] loss: 0.883
Epoch: 2 -> Loss: 0.775784909725
Epoch: 2 -> Test Accuracy: 69.0
[3, 60] loss: 0.800
[3, 120] loss: 0.807
[3, 180] loss: 0.783
[3, 240] loss: 0.803
[3, 300] loss: 0.791
[3, 360] loss: 0.775
Epoch: 3 -> Loss: 0.804202079773
Epoch: 3 -> Test Accuracy: 72.79
[4, 60] loss: 0.708
[4, 120] loss: 0.718
[4, 180] loss: 0.737
[4, 240] loss: 0.714
[4, 300] loss: 0.705
[4, 360] loss: 0.708
Epoch: 4 -> Loss: 0.555963754654
Epoch: 4 -> Test Accuracy: 75.21
[5, 60] loss: 0.648
[5, 120] loss: 0.670
[5, 180] loss: 0.670
[5, 240] loss: 0.658
[5, 300] loss: 0.669
[5, 360] loss: 0.655
Epoch: 5 -> Loss: 0.522774040699
Epoch: 5 -> Test Accuracy: 75.59
[6, 60] loss: 0.632
[6, 120] loss: 0.638
[6, 1

Epoch: 42 -> Loss: 0.593099057674
Epoch: 42 -> Test Accuracy: 83.77
[43, 60] loss: 0.380
[43, 120] loss: 0.389
[43, 180] loss: 0.401
[43, 240] loss: 0.431
[43, 300] loss: 0.406
[43, 360] loss: 0.399
Epoch: 43 -> Loss: 0.20330825448
Epoch: 43 -> Test Accuracy: 83.59
[44, 60] loss: 0.359
[44, 120] loss: 0.397
[44, 180] loss: 0.400
[44, 240] loss: 0.403
[44, 300] loss: 0.405
[44, 360] loss: 0.407
Epoch: 44 -> Loss: 0.428490787745
Epoch: 44 -> Test Accuracy: 84.12
[45, 60] loss: 0.389
[45, 120] loss: 0.412
[45, 180] loss: 0.382
[45, 240] loss: 0.397
[45, 300] loss: 0.401
[45, 360] loss: 0.413
Epoch: 45 -> Loss: 0.486002355814
Epoch: 45 -> Test Accuracy: 83.85
[46, 60] loss: 0.398
[46, 120] loss: 0.382
[46, 180] loss: 0.416
[46, 240] loss: 0.384
[46, 300] loss: 0.406
[46, 360] loss: 0.422
Epoch: 46 -> Loss: 0.473516404629
Epoch: 46 -> Test Accuracy: 82.9
[47, 60] loss: 0.363
[47, 120] loss: 0.402
[47, 180] loss: 0.396
[47, 240] loss: 0.389
[47, 300] loss: 0.410
[47, 360] loss: 0.407
Epoch: 

[84, 60] loss: 0.114
[84, 120] loss: 0.121
[84, 180] loss: 0.132
[84, 240] loss: 0.132
[84, 300] loss: 0.142
[84, 360] loss: 0.158
Epoch: 84 -> Loss: 0.0979826822877
Epoch: 84 -> Test Accuracy: 88.32
[85, 60] loss: 0.120
[85, 120] loss: 0.130
[85, 180] loss: 0.140
[85, 240] loss: 0.146
[85, 300] loss: 0.142
[85, 360] loss: 0.143
Epoch: 85 -> Loss: 0.166171133518
Epoch: 85 -> Test Accuracy: 88.5
[86, 60] loss: 0.119
[86, 120] loss: 0.122
[86, 180] loss: 0.136
[86, 240] loss: 0.129
[86, 300] loss: 0.135
[86, 360] loss: 0.156
Epoch: 86 -> Loss: 0.151281192899
Epoch: 86 -> Test Accuracy: 88.61
[87, 60] loss: 0.120
[87, 120] loss: 0.109
[87, 180] loss: 0.132
[87, 240] loss: 0.128
[87, 300] loss: 0.143
[87, 360] loss: 0.142
Epoch: 87 -> Loss: 0.173461005092
Epoch: 87 -> Test Accuracy: 88.67
[88, 60] loss: 0.116
[88, 120] loss: 0.128
[88, 180] loss: 0.130
[88, 240] loss: 0.139
[88, 300] loss: 0.140
[88, 360] loss: 0.140
Epoch: 88 -> Loss: 0.225553706288
Epoch: 88 -> Test Accuracy: 88.07
[89, 

[124, 180] loss: 0.021
[124, 240] loss: 0.020
[124, 300] loss: 0.020
[124, 360] loss: 0.022
Epoch: 124 -> Loss: 0.00913160480559
Epoch: 124 -> Test Accuracy: 91.3
[125, 60] loss: 0.019
[125, 120] loss: 0.019
[125, 180] loss: 0.019
[125, 240] loss: 0.020
[125, 300] loss: 0.020
[125, 360] loss: 0.021
Epoch: 125 -> Loss: 0.0156837590039
Epoch: 125 -> Test Accuracy: 91.25
[126, 60] loss: 0.020
[126, 120] loss: 0.019
[126, 180] loss: 0.019
[126, 240] loss: 0.018
[126, 300] loss: 0.015
[126, 360] loss: 0.020
Epoch: 126 -> Loss: 0.00777263054624
Epoch: 126 -> Test Accuracy: 91.32
[127, 60] loss: 0.016
[127, 120] loss: 0.017
[127, 180] loss: 0.016
[127, 240] loss: 0.016
[127, 300] loss: 0.018
[127, 360] loss: 0.017
Epoch: 127 -> Loss: 0.0105040315539
Epoch: 127 -> Test Accuracy: 91.21
[128, 60] loss: 0.016
[128, 120] loss: 0.016
[128, 180] loss: 0.016
[128, 240] loss: 0.015
[128, 300] loss: 0.015
[128, 360] loss: 0.015
Epoch: 128 -> Loss: 0.0326036699116
Epoch: 128 -> Test Accuracy: 91.28
[129

Epoch: 163 -> Loss: 0.012463092804
Epoch: 163 -> Test Accuracy: 91.39
[164, 60] loss: 0.006
[164, 120] loss: 0.007
[164, 180] loss: 0.007
[164, 240] loss: 0.006
[164, 300] loss: 0.007
[164, 360] loss: 0.007
Epoch: 164 -> Loss: 0.03215944767
Epoch: 164 -> Test Accuracy: 91.28
[165, 60] loss: 0.007
[165, 120] loss: 0.006
[165, 180] loss: 0.007
[165, 240] loss: 0.006
[165, 300] loss: 0.007
[165, 360] loss: 0.007
Epoch: 165 -> Loss: 0.0114584919065
Epoch: 165 -> Test Accuracy: 91.31
[166, 60] loss: 0.007
[166, 120] loss: 0.006
[166, 180] loss: 0.006
[166, 240] loss: 0.007
[166, 300] loss: 0.007
[166, 360] loss: 0.007
Epoch: 166 -> Loss: 0.00599935650826
Epoch: 166 -> Test Accuracy: 91.39
[167, 60] loss: 0.007
[167, 120] loss: 0.006
[167, 180] loss: 0.007
[167, 240] loss: 0.006
[167, 300] loss: 0.006
[167, 360] loss: 0.007
Epoch: 167 -> Loss: 0.00732953567058
Epoch: 167 -> Test Accuracy: 91.35
[168, 60] loss: 0.006
[168, 120] loss: 0.007
[168, 180] loss: 0.006
[168, 240] loss: 0.006
[168, 3

In [11]:
# save variables
fm.save_variable([class_NIN_loss_log, class_NIN_test_accuracy_log], "supervised_NIN")

# Semi-supervised Learning

In [24]:
# initialize networks
semi_net = fm.load_net("RotNet_rotation_200_4_block_net")

In [25]:
semi_loss_log, semi_accuracy_log, super_loss_log, super_accuracy_log = tr.train_semi([20, 100, 400, 1000, 5000], 10, 
    trainset, testset, 128, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], [0.1, 0.02, 0.004, 0.0008],
               [60, 120, 160, 200], 0.9, 5e-4, semi_net, criterion)

Epoch: 1 -> Loss: 2.1992418766
Epoch: 1 -> Test Accuracy: 40.72
Epoch: 2 -> Loss: 1.25360822678
Epoch: 2 -> Test Accuracy: 50.21
Epoch: 3 -> Loss: 0.949498295784
Epoch: 3 -> Test Accuracy: 53.53
Epoch: 4 -> Loss: 0.62757062912
Epoch: 4 -> Test Accuracy: 55.36
Epoch: 5 -> Loss: 0.481527328491
Epoch: 5 -> Test Accuracy: 57.63
Epoch: 6 -> Loss: 0.460844159126
Epoch: 6 -> Test Accuracy: 58.93
Epoch: 7 -> Loss: 0.318598896265
Epoch: 7 -> Test Accuracy: 60.19
Epoch: 8 -> Loss: 0.213707759976
Epoch: 8 -> Test Accuracy: 60.57
Epoch: 9 -> Loss: 0.180244058371
Epoch: 9 -> Test Accuracy: 60.78
Epoch: 10 -> Loss: 0.0976581647992
Epoch: 10 -> Test Accuracy: 60.77
Epoch: 11 -> Loss: 0.118126921356
Epoch: 11 -> Test Accuracy: 60.87
Epoch: 12 -> Loss: 0.0888189077377
Epoch: 12 -> Test Accuracy: 61.2
Epoch: 13 -> Loss: 0.0779830217361
Epoch: 13 -> Test Accuracy: 61.22
Epoch: 14 -> Loss: 0.0497829467058
Epoch: 14 -> Test Accuracy: 61.35
Epoch: 15 -> Loss: 0.0414481908083
Epoch: 15 -> Test Accuracy: 61.1

Epoch: 20 -> Test Accuracy: 28.75
Epoch: 21 -> Loss: 0.948741614819
Epoch: 21 -> Test Accuracy: 29.45
Epoch: 22 -> Loss: 0.931711852551
Epoch: 22 -> Test Accuracy: 30.44
Epoch: 23 -> Loss: 0.869188129902
Epoch: 23 -> Test Accuracy: 30.7
Epoch: 24 -> Loss: 0.583420753479
Epoch: 24 -> Test Accuracy: 31.01
Epoch: 25 -> Loss: 0.650334239006
Epoch: 25 -> Test Accuracy: 31.08
Epoch: 26 -> Loss: 0.495277136564
Epoch: 26 -> Test Accuracy: 29.84
Epoch: 27 -> Loss: 0.530931055546
Epoch: 27 -> Test Accuracy: 30.49
Epoch: 28 -> Loss: 0.479001671076
Epoch: 28 -> Test Accuracy: 30.51
Epoch: 29 -> Loss: 0.338510364294
Epoch: 29 -> Test Accuracy: 31.03
Epoch: 30 -> Loss: 0.475204825401
Epoch: 30 -> Test Accuracy: 33.27
Epoch: 31 -> Loss: 0.409424722195
Epoch: 31 -> Test Accuracy: 30.28
Epoch: 32 -> Loss: 0.349939644337
Epoch: 32 -> Test Accuracy: 31.13
Epoch: 33 -> Loss: 0.40071734786
Epoch: 33 -> Test Accuracy: 28.65
Epoch: 34 -> Loss: 0.376473635435
Epoch: 34 -> Test Accuracy: 31.01
Epoch: 35 -> Los

Epoch: 138 -> Test Accuracy: 33.5
Epoch: 139 -> Loss: 0.00422385660931
Epoch: 139 -> Test Accuracy: 33.48
Epoch: 140 -> Loss: 0.00757612101734
Epoch: 140 -> Test Accuracy: 33.45
Epoch: 141 -> Loss: 0.00782237481326
Epoch: 141 -> Test Accuracy: 33.46
Epoch: 142 -> Loss: 0.00492163514718
Epoch: 142 -> Test Accuracy: 33.48
Epoch: 143 -> Loss: 0.0108833573759
Epoch: 143 -> Test Accuracy: 33.46
Epoch: 144 -> Loss: 0.00434049637988
Epoch: 144 -> Test Accuracy: 33.47
Epoch: 145 -> Loss: 0.00675307400525
Epoch: 145 -> Test Accuracy: 33.51
Epoch: 146 -> Loss: 0.00457634544
Epoch: 146 -> Test Accuracy: 33.49
Epoch: 147 -> Loss: 0.00417333841324
Epoch: 147 -> Test Accuracy: 33.47
Epoch: 148 -> Loss: 0.00461537949741
Epoch: 148 -> Test Accuracy: 33.48
Epoch: 149 -> Loss: 0.00940836127847
Epoch: 149 -> Test Accuracy: 33.47
Epoch: 150 -> Loss: 0.0037887361832
Epoch: 150 -> Test Accuracy: 33.43
Epoch: 151 -> Loss: 0.00704176537693
Epoch: 151 -> Test Accuracy: 33.44
Epoch: 152 -> Loss: 0.0058347051963

Epoch: 55 -> Loss: 0.00788806937635
Epoch: 55 -> Test Accuracy: 71.2
Epoch: 56 -> Loss: 0.00546456314623
Epoch: 56 -> Test Accuracy: 71.17
Epoch: 57 -> Loss: 0.0255545191467
Epoch: 57 -> Test Accuracy: 71.11
Epoch: 58 -> Loss: 0.0102572627366
Epoch: 58 -> Test Accuracy: 71.17
Epoch: 59 -> Loss: 0.00920040346682
Epoch: 59 -> Test Accuracy: 71.09
Epoch: 60 -> Loss: 0.00655400753021
Epoch: 60 -> Test Accuracy: 71.15
Epoch: 61 -> Loss: 0.00766201643273
Epoch: 61 -> Test Accuracy: 71.26
Epoch: 62 -> Loss: 0.0107051227242
Epoch: 62 -> Test Accuracy: 71.26
Epoch: 63 -> Loss: 0.00476559763774
Epoch: 63 -> Test Accuracy: 71.34
Epoch: 64 -> Loss: 0.00601513590664
Epoch: 64 -> Test Accuracy: 71.32
Epoch: 65 -> Loss: 0.00635406142101
Epoch: 65 -> Test Accuracy: 71.26
Epoch: 66 -> Loss: 0.00865513086319
Epoch: 66 -> Test Accuracy: 71.24
Epoch: 67 -> Loss: 0.0061973836273
Epoch: 67 -> Test Accuracy: 71.32
Epoch: 68 -> Loss: 0.00733599998057
Epoch: 68 -> Test Accuracy: 71.36
Epoch: 69 -> Loss: 0.0051

Epoch: 74 -> Test Accuracy: 53.12
Epoch: 75 -> Loss: 0.0133948875591
Epoch: 75 -> Test Accuracy: 52.85
Epoch: 76 -> Loss: 0.00839868374169
Epoch: 76 -> Test Accuracy: 52.87
Epoch: 77 -> Loss: 0.0084963394329
Epoch: 77 -> Test Accuracy: 53.22
Epoch: 78 -> Loss: 0.00737501587719
Epoch: 78 -> Test Accuracy: 53.17
Epoch: 79 -> Loss: 0.0110773723572
Epoch: 79 -> Test Accuracy: 53.25
Epoch: 80 -> Loss: 0.00879410561174
Epoch: 80 -> Test Accuracy: 53.26
Epoch: 81 -> Loss: 0.0130020566285
Epoch: 81 -> Test Accuracy: 53.1
Epoch: 82 -> Loss: 0.00690402416512
Epoch: 82 -> Test Accuracy: 52.89
Epoch: 83 -> Loss: 0.0114755583927
Epoch: 83 -> Test Accuracy: 53.21
Epoch: 84 -> Loss: 0.00637876056135
Epoch: 84 -> Test Accuracy: 53.21
Epoch: 85 -> Loss: 0.00438362360001
Epoch: 85 -> Test Accuracy: 53.21
Epoch: 86 -> Loss: 0.0122922156006
Epoch: 86 -> Test Accuracy: 53.27
Epoch: 87 -> Loss: 0.00887689180672
Epoch: 87 -> Test Accuracy: 53.37
Epoch: 88 -> Loss: 0.0137259000912
Epoch: 88 -> Test Accuracy: 

...

[28, 60] loss: 0.411
[28, 120] loss: 0.425
[28, 180] loss: 0.439
[28, 240] loss: 0.427
[28, 300] loss: 0.415
[28, 360] loss: 0.461
Epoch: 28 -> Loss: 0.368227303028
Epoch: 28 -> Test Accuracy: 82.59
[29, 60] loss: 0.428
[29, 120] loss: 0.409
[29, 180] loss: 0.431
[29, 240] loss: 0.432
[29, 300] loss: 0.439
[29, 360] loss: 0.434
Epoch: 29 -> Loss: 0.340874284506
Epoch: 29 -> Test Accuracy: 81.51
[30, 60] loss: 0.396
[30, 120] loss: 0.419
[30, 180] loss: 0.418
[30, 240] loss: 0.427
[30, 300] loss: 0.440
[30, 360] loss: 0.437
Epoch: 30 -> Loss: 0.372979491949
Epoch: 30 -> Test Accuracy: 83.53
[31, 60] loss: 0.378
[31, 120] loss: 0.410
[31, 180] loss: 0.416
[31, 240] loss: 0.445
[31, 300] loss: 0.414
[31, 360] loss: 0.425
Epoch: 31 -> Loss: 0.482057899237
Epoch: 31 -> Test Accuracy: 82.84
[32, 60] loss: 0.415
[32, 120] loss: 0.408
[32, 180] loss: 0.414
[32, 240] loss: 0.428
[32, 300] loss: 0.438
[32, 360] loss: 0.408
Epoch: 32 -> Loss: 0.370775550604
Epoch: 32 -> Test Accuracy: 82.93
[33, 

[69, 180] loss: 0.124
[69, 240] loss: 0.133
[69, 300] loss: 0.142
[69, 360] loss: 0.139
Epoch: 69 -> Loss: 0.21241530776
Epoch: 69 -> Test Accuracy: 89.57
[70, 60] loss: 0.123
[70, 120] loss: 0.116
[70, 180] loss: 0.128
[70, 240] loss: 0.125
[70, 300] loss: 0.139
[70, 360] loss: 0.139
Epoch: 70 -> Loss: 0.166151300073
Epoch: 70 -> Test Accuracy: 88.83
[71, 60] loss: 0.113
[71, 120] loss: 0.130
[71, 180] loss: 0.125
[71, 240] loss: 0.143
[71, 300] loss: 0.140
[71, 360] loss: 0.144
Epoch: 71 -> Loss: 0.167607396841
Epoch: 71 -> Test Accuracy: 88.95
[72, 60] loss: 0.131
[72, 120] loss: 0.135
[72, 180] loss: 0.137
[72, 240] loss: 0.124
[72, 300] loss: 0.149
[72, 360] loss: 0.158
Epoch: 72 -> Loss: 0.0815150886774
Epoch: 72 -> Test Accuracy: 88.72
[73, 60] loss: 0.118
[73, 120] loss: 0.121
[73, 180] loss: 0.130
[73, 240] loss: 0.139
[73, 300] loss: 0.136
[73, 360] loss: 0.145
Epoch: 73 -> Loss: 0.083897896111
Epoch: 73 -> Test Accuracy: 89.06
[74, 60] loss: 0.113
[74, 120] loss: 0.112
[74, 

[110, 120] loss: 0.111
[110, 180] loss: 0.115
[110, 240] loss: 0.114
[110, 300] loss: 0.126
[110, 360] loss: 0.136
Epoch: 110 -> Loss: 0.198725804687
Epoch: 110 -> Test Accuracy: 89.22
[111, 60] loss: 0.100
[111, 120] loss: 0.117
[111, 180] loss: 0.113
[111, 240] loss: 0.127
[111, 300] loss: 0.149
[111, 360] loss: 0.139
Epoch: 111 -> Loss: 0.181658118963
Epoch: 111 -> Test Accuracy: 88.22
[112, 60] loss: 0.112
[112, 120] loss: 0.118
[112, 180] loss: 0.120
[112, 240] loss: 0.114
[112, 300] loss: 0.131
[112, 360] loss: 0.128
Epoch: 112 -> Loss: 0.162451073527
Epoch: 112 -> Test Accuracy: 89.05
[113, 60] loss: 0.110
[113, 120] loss: 0.113
[113, 180] loss: 0.111
[113, 240] loss: 0.116
[113, 300] loss: 0.128
[113, 360] loss: 0.129
Epoch: 113 -> Loss: 0.118588209152
Epoch: 113 -> Test Accuracy: 89.06
[114, 60] loss: 0.106
[114, 120] loss: 0.113
[114, 180] loss: 0.116
[114, 240] loss: 0.124
[114, 300] loss: 0.128
[114, 360] loss: 0.134
Epoch: 114 -> Loss: 0.142578884959
Epoch: 114 -> Test Acc

[149, 360] loss: 0.008
Epoch: 149 -> Loss: 0.00937450490892
Epoch: 149 -> Test Accuracy: 91.6
[150, 60] loss: 0.008
[150, 120] loss: 0.009
[150, 180] loss: 0.009
[150, 240] loss: 0.009
[150, 300] loss: 0.009
[150, 360] loss: 0.009
Epoch: 150 -> Loss: 0.0093620121479
Epoch: 150 -> Test Accuracy: 91.55
[151, 60] loss: 0.009
[151, 120] loss: 0.008
[151, 180] loss: 0.009
[151, 240] loss: 0.010
[151, 300] loss: 0.010
[151, 360] loss: 0.010
Epoch: 151 -> Loss: 0.0233277864754
Epoch: 151 -> Test Accuracy: 91.83
[152, 60] loss: 0.010
[152, 120] loss: 0.008
[152, 180] loss: 0.009
[152, 240] loss: 0.010
[152, 300] loss: 0.010
[152, 360] loss: 0.009
Epoch: 152 -> Loss: 0.00585313420743
Epoch: 152 -> Test Accuracy: 91.83
[153, 60] loss: 0.009
[153, 120] loss: 0.009
[153, 180] loss: 0.008
[153, 240] loss: 0.009
[153, 300] loss: 0.009
[153, 360] loss: 0.010
Epoch: 153 -> Loss: 0.0230761114508
Epoch: 153 -> Test Accuracy: 91.78
[154, 60] loss: 0.008
[154, 120] loss: 0.009
[154, 180] loss: 0.009
[154,

[189, 60] loss: 0.006
[189, 120] loss: 0.006
[189, 180] loss: 0.006
[189, 240] loss: 0.007
[189, 300] loss: 0.006
[189, 360] loss: 0.006
Epoch: 189 -> Loss: 0.00674162525684
Epoch: 189 -> Test Accuracy: 91.64
[190, 60] loss: 0.006
[190, 120] loss: 0.006
[190, 180] loss: 0.006
[190, 240] loss: 0.006
[190, 300] loss: 0.007
[190, 360] loss: 0.006
Epoch: 190 -> Loss: 0.00866503734142
Epoch: 190 -> Test Accuracy: 91.63
[191, 60] loss: 0.006
[191, 120] loss: 0.007
[191, 180] loss: 0.007
[191, 240] loss: 0.006
[191, 300] loss: 0.006
[191, 360] loss: 0.006
Epoch: 191 -> Loss: 0.00852023996413
Epoch: 191 -> Test Accuracy: 91.59
[192, 60] loss: 0.006
[192, 120] loss: 0.006
[192, 180] loss: 0.007
[192, 240] loss: 0.006
[192, 300] loss: 0.007
[192, 360] loss: 0.006
Epoch: 192 -> Loss: 0.0124972881749
Epoch: 192 -> Test Accuracy: 91.74
[193, 60] loss: 0.005
[193, 120] loss: 0.006
[193, 180] loss: 0.007
[193, 240] loss: 0.006
[193, 300] loss: 0.006
[193, 360] loss: 0.006
Epoch: 193 -> Loss: 0.009459

In [26]:
# save variables
fm.save_variable([semi_loss_log, semi_accuracy_log, super_loss_log, super_accuracy_log], "semi-supervised")

# Evaluate Test Accuracies

In [6]:
# 3 ConvBlock RotNet model and Classifiers
ev.evaluate_all(3, testloader, classes)

Evaluating RotNet model with 3 Convolutional Blocks:


Evaluating Rotation Task:
Test Accuracy:  92.190 %
Accuracy per class:
Test Accuracy of original :  92.100 %
Test Accuracy of 90 rotation :  92.110 %
Test Accuracy of 180 rotation :  92.140 %
Test Accuracy of 270 rotation :  92.410 %


--------------------------------------------------------------------------------


Starting to evaluate Non-Linear Classifier:


Evaluating Non-Linear Classifier on Convolutional Block 1:
Test Accuracy:  83.280 %
Accuracy per class:
Test Accuracy of plane :  85.100 %
Test Accuracy of car :  91.400 %
Test Accuracy of bird :  76.900 %
Test Accuracy of cat :  69.000 %
Test Accuracy of deer :  79.700 %
Test Accuracy of dog :  72.900 %
Test Accuracy of frog :  89.500 %
Test Accuracy of horse :  86.300 %
Test Accuracy of ship :  91.100 %
Test Accuracy of truck :  90.900 %


Evaluating Non-Linear Classifier on Convolutional Block 2:
Test Accuracy:  86.510 %
Accuracy per class:
Test Accuracy of plane :  87.3

{'Accuracy ConvClassifier ConvBlock 1': 86.85,
 'Accuracy ConvClassifier ConvBlock 2': 88.82,
 'Accuracy ConvClassifier ConvBlock 3': 61.91,
 'Accuracy Non-Linear ConvBlock 1': 83.28,
 'Accuracy Non-Linear ConvBlock 2': 86.51,
 'Accuracy Non-Linear ConvBlock 3': 54.07,
 'Accuracy Rotation Task': 92.19,
 'Class Accuracy ConvClassifier ConvBlock 1': [88.3,
  93.9,
  80.2,
  76.9,
  86.7,
  78.4,
  92.5,
  89.2,
  91.5,
  90.9],
 'Class Accuracy ConvClassifier ConvBlock 2': [89.2,
  94.0,
  84.4,
  79.9,
  89.8,
  83.4,
  92.2,
  90.0,
  93.3,
  92.0],
 'Class Accuracy ConvClassifier ConvBlock 3': [64.7,
  64.5,
  51.1,
  49.3,
  61.2,
  55.3,
  71.6,
  61.4,
  67.9,
  72.1],
 'Class Accuracy Non-Linear ConvBlock 1': [85.1,
  91.4,
  76.9,
  69.0,
  79.7,
  72.9,
  89.5,
  86.3,
  91.1,
  90.9],
 'Class Accuracy Non-Linear ConvBlock 2': [87.3,
  92.6,
  82.4,
  77.1,
  86.6,
  77.6,
  90.4,
  88.6,
  92.4,
  90.1],
 'Class Accuracy Non-Linear ConvBlock 3': [61.8,
  59.2,
  43.1,
  36.5,
 

In [7]:
# 4 ConvBlock RotNet model and Classifiers
ev.evaluate_all(4, testloader, classes)

Evaluating RotNet model with 4 Convolutional Blocks:


Evaluating Rotation Task:
Test Accuracy:  92.013 %
Accuracy per class:
Test Accuracy of original :  91.790 %
Test Accuracy of 90 rotation :  92.080 %
Test Accuracy of 180 rotation :  91.950 %
Test Accuracy of 270 rotation :  92.230 %


--------------------------------------------------------------------------------


Starting to evaluate Non-Linear Classifier:


Evaluating Non-Linear Classifier on Convolutional Block 1:
Test Accuracy:  82.730 %
Accuracy per class:
Test Accuracy of plane :  85.700 %
Test Accuracy of car :  87.800 %
Test Accuracy of bird :  76.200 %
Test Accuracy of cat :  68.400 %
Test Accuracy of deer :  80.100 %
Test Accuracy of dog :  72.900 %
Test Accuracy of frog :  89.100 %
Test Accuracy of horse :  87.800 %
Test Accuracy of ship :  90.900 %
Test Accuracy of truck :  88.400 %


Evaluating Non-Linear Classifier on Convolutional Block 2:
Test Accuracy:  85.530 %
Accuracy per class:
Test Accuracy of plane :  87.6

{'Accuracy ConvClassifier ConvBlock 1': 85.19,
 'Accuracy ConvClassifier ConvBlock 2': 87.51,
 'Accuracy ConvClassifier ConvBlock 3': 82.66,
 'Accuracy ConvClassifier ConvBlock 4': 50.91,
 'Accuracy Non-Linear ConvBlock 1': 82.73,
 'Accuracy Non-Linear ConvBlock 2': 85.53,
 'Accuracy Non-Linear ConvBlock 3': 80.99,
 'Accuracy Non-Linear ConvBlock 4': 43.2,
 'Accuracy Rotation Task': 92.0125,
 'Class Accuracy ConvClassifier ConvBlock 1': [86.0,
  92.5,
  79.2,
  73.5,
  82.5,
  79.7,
  90.4,
  86.8,
  91.3,
  90.0],
 'Class Accuracy ConvClassifier ConvBlock 2': [89.4,
  92.0,
  83.1,
  78.3,
  88.9,
  80.5,
  91.0,
  90.4,
  91.2,
  90.3],
 'Class Accuracy ConvClassifier ConvBlock 3': [83.5,
  86.6,
  76.0,
  74.5,
  83.0,
  74.9,
  88.3,
  85.0,
  87.0,
  87.8],
 'Class Accuracy ConvClassifier ConvBlock 4': [60.5,
  53.5,
  39.5,
  38.0,
  36.6,
  49.2,
  65.0,
  52.6,
  63.0,
  51.2],
 'Class Accuracy Non-Linear ConvBlock 1': [85.7,
  87.8,
  76.2,
  68.4,
  80.1,
  72.9,
  89.1,
  87

In [8]:
# 5 ConvBlock RotNet model and Classifiers
ev.evaluate_all(5, testloader, classes)

Evaluating RotNet model with 5 Convolutional Blocks:


Evaluating Rotation Task:
Test Accuracy:  92.088 %
Accuracy per class:
Test Accuracy of original :  92.380 %
Test Accuracy of 90 rotation :  92.150 %
Test Accuracy of 180 rotation :  91.850 %
Test Accuracy of 270 rotation :  91.970 %


--------------------------------------------------------------------------------


Starting to evaluate Non-Linear Classifier:


Evaluating Non-Linear Classifier on Convolutional Block 1:
Test Accuracy:  82.990 %
Accuracy per class:
Test Accuracy of plane :  82.700 %
Test Accuracy of car :  90.200 %
Test Accuracy of bird :  77.800 %
Test Accuracy of cat :  68.300 %
Test Accuracy of deer :  79.100 %
Test Accuracy of dog :  73.500 %
Test Accuracy of frog :  88.600 %
Test Accuracy of horse :  88.000 %
Test Accuracy of ship :  91.800 %
Test Accuracy of truck :  89.900 %


Evaluating Non-Linear Classifier on Convolutional Block 2:
Test Accuracy:  86.610 %
Accuracy per class:
Test Accuracy of plane :  88.9

{'Accuracy ConvClassifier ConvBlock 1': 86.15,
 'Accuracy ConvClassifier ConvBlock 2': 88.37,
 'Accuracy ConvClassifier ConvBlock 3': 85.15,
 'Accuracy ConvClassifier ConvBlock 4': 72.95,
 'Accuracy ConvClassifier ConvBlock 5': 41.48,
 'Accuracy Non-Linear ConvBlock 1': 82.99,
 'Accuracy Non-Linear ConvBlock 2': 86.61,
 'Accuracy Non-Linear ConvBlock 3': 82.97,
 'Accuracy Non-Linear ConvBlock 4': 69.83,
 'Accuracy Non-Linear ConvBlock 5': 36.9,
 'Accuracy Rotation Task': 92.0875,
 'Class Accuracy ConvClassifier ConvBlock 1': [89.5,
  93.0,
  79.1,
  73.0,
  86.4,
  79.9,
  90.7,
  88.3,
  90.7,
  90.9],
 'Class Accuracy ConvClassifier ConvBlock 2': [88.9,
  92.7,
  84.4,
  79.7,
  87.9,
  81.5,
  91.5,
  88.9,
  94.3,
  93.9],
 'Class Accuracy ConvClassifier ConvBlock 3': [86.8,
  90.7,
  79.0,
  76.6,
  84.9,
  76.1,
  89.8,
  87.4,
  90.5,
  89.7],
 'Class Accuracy ConvClassifier ConvBlock 4': [78.4,
  77.3,
  63.7,
  61.5,
  72.9,
  65.0,
  81.6,
  75.7,
  76.1,
  77.3],
 'Class Acc

In [6]:
# Supervised NIN
ev.evaluate_all(0, testloader, classes)

Evaluating Supervised NIN Classification Task:
Test Accuracy:  91.390 %
Test Accuracy of plane :  91.600 %
Test Accuracy of car :  95.900 %
Test Accuracy of bird :  87.000 %
Test Accuracy of cat :  83.200 %
Test Accuracy of deer :  92.100 %
Test Accuracy of dog :  85.500 %
Test Accuracy of frog :  94.800 %
Test Accuracy of horse :  93.900 %
Test Accuracy of ship :  94.700 %
Test Accuracy of truck :  95.200 %


{'Accuracy Supervised NIN': 91.39,
 'Class Accuracy Supervised NIN': [91.6,
  95.9,
  87.0,
  83.2,
  92.1,
  85.5,
  94.8,
  93.9,
  94.7,
  95.2]}

In [6]:
# semi-supervised
ev.evaluate_all(-1, testloader, classes, semi=[20, 100, 400, 1000, 5000])

Evaluating Semi-supervised Learning Experiment:


Evaluating Semi-supervised experiment with 20 images per class:
Test Accuracy:  61.670 %
Test Accuracy of plane :  59.500 %
Test Accuracy of car :  72.100 %
Test Accuracy of bird :  44.700 %
Test Accuracy of cat :  42.600 %
Test Accuracy of deer :  57.200 %
Test Accuracy of dog :  60.100 %
Test Accuracy of frog :  77.800 %
Test Accuracy of horse :  59.600 %
Test Accuracy of ship :  67.300 %
Test Accuracy of truck :  75.800 %


Evaluating supervised NIN experiment with 20 images per class:
Test Accuracy:  11.560 %
Test Accuracy of plane :  39.000 %
Test Accuracy of car :  39.400 %
Test Accuracy of bird :  20.100 %
Test Accuracy of cat :  23.500 %
Test Accuracy of deer :  26.500 %
Test Accuracy of dog :  25.100 %
Test Accuracy of frog :  42.700 %
Test Accuracy of horse :  38.100 %
Test Accuracy of ship :  44.000 %
Test Accuracy of truck :  35.800 %


--------------------------------------------------------------------------------




Eval

{'Accuracy Semi-supervised 100': 71.44,
 'Accuracy Semi-supervised 1000': 82.37,
 'Accuracy Semi-supervised 20': 61.67,
 'Accuracy Semi-supervised 400': 78.86,
 'Accuracy Semi-supervised 5000': 87.56,
 'Accuracy Supervised NIN 100': 9.69,
 'Accuracy Supervised NIN 1000': 10.04,
 'Accuracy Supervised NIN 20': 11.56,
 'Accuracy Supervised NIN 400': 9.83,
 'Accuracy Supervised NIN 5000': 9.94,
 'Class Accuracy Semi-supervised 100': [70.5,
  82.2,
  58.0,
  58.2,
  65.9,
  66.6,
  79.9,
  73.5,
  78.2,
  81.4],
 'Class Accuracy Semi-supervised 1000': [82.8,
  89.0,
  77.0,
  71.5,
  81.9,
  74.4,
  88.0,
  83.3,
  87.7,
  88.1],
 'Class Accuracy Semi-supervised 20': [59.5,
  72.1,
  44.7,
  42.6,
  57.2,
  60.1,
  77.8,
  59.6,
  67.3,
  75.8],
 'Class Accuracy Semi-supervised 400': [79.1,
  88.7,
  71.2,
  64.6,
  78.1,
  71.3,
  86.1,
  80.8,
  85.8,
  82.9],
 'Class Accuracy Semi-supervised 5000': [88.4,
  92.9,
  81.8,
  79.2,
  89.0,
  81.4,
  92.0,
  89.1,
  91.8,
  90.0],
 'Class Ac

# Plots

In [ ]:
p.plot_all()